In [ ]:
import pandas as pd
#import gseapy as gp
import matplotlib.pyplot as plt

from goatools import obo_parser
import wget
import os

import seaborn as sns
import mygene
import numpy as np
import collections
import pycrosstalker

In [ ]:
which_os = "windows11"

In [ ]:
#RCC = pd.read_csv("/home/larissa/Documents/Masterarbeit/RCC_results/liana_log_norm/no_subset_glom_removed/T_lr_ready.csv", index_col=0)
#MF = pd.read_csv("/home/larissa/Documents/Masterarbeit/MF_results/liana/all_celltypes/MF_lr_ready.csv", index_col=0)

#RCC = pd.read_csv("D:\\studium\\Masterarbeit\\RCC_results\\liana_log_norm\\no_subset_glom_removed\\T_lr_ready.csv", index_col=0)
#MF = pd.read_csv("D:\\studium\\Masterarbeit\\MF_results\\liana\\all_celltypes\\MF_lr_ready.csv", index_col=0)


RCC = pd.read_csv("C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\RCC_results\\liana_log_norm\\no_subset_glom_removed\\T_lr_ready.csv", index_col = 0)
MF = pd.read_csv("C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\MF_results\\liana\\all_celltypes\\MF_lr_ready.csv", index_col = 0)


RCC["lr_pair"] = RCC["gene_A"] + "_" + RCC["gene_B"]
MF["lr_pair"] = MF["gene_A"] + "_" + MF["gene_B"]

RCC["cluster_pair"] = RCC["source"] + "@" + RCC["target"]
MF["cluster_pair"] = MF["source"] + "@" + MF["target"]

RCC = RCC[~RCC.apply(lambda r: r.str.contains('Glomerular endothelium').any(), axis=1)]

In [ ]:
#RBConfig res 1.25 after removing glom endo
cluster_dict = {"Monocytes" : ["MF_CD16_monocytes", "MF_Classical_monocytes", "MF_EBM_prog", "MF_GMPs",
    "MF_Inflammatory_CD14_monocytes", "MF_Late_Neutro_Prog", "MF_Macrophages",
    "MF_Neutrophils", "MF_SPP1_Macrophages", "MF_cDC1", "MF_cDC2", "RCC_Classical monocytes", "RCC_Non-classical monocytes", "RCC_TAM 1", "RCC_TAM 2", "RCC_TAM 3", "RCC_TAM 4"],
"Lymphocytes" : ["MF_B_cells", "MF_CD8_T_cells","MF_NK_T_cells", "RCC_B cells",
    "RCC_CD8 T cells", "RCC_Cytotoxic T cells", "RCC_IGHG-high plasma cells",
    "RCC_NK cells", "RCC_Plasma cells", "RCC_Regulatory T cells",
    "RCC_Resting/memory T cells"],#, "RCC_Proximal tubule"],
"Tumor_prog" : ["MF_B_cell_prog", "MF_Plasma_cells",
                 "MF_Early_Neutro_Prog", "MF_HSPCs", "MF_MEPs", "MF_MK_prog",
    "RCC_DCT/CNT", "RCC_Epithelial progenitor-like cells", "RCC_Principal cells",
    "RCC_Tumor cells 1", "RCC_Tumor cells 2", "RCC_Tumor cells 3", "RCC_tAL of LOH"],
"Endothelial" : [ "MF_pDCs", "MF_Arterial_EC", "MF_Erythroid_cells", "MF_Late_Erythroid_Prog",
    "MF_Sinusoidal_EC", "RCC_AVR", "RCC_DVR", "RCC_Tumor AVR-like vasculature",
    "RCC_Tumor vasculature 1", "RCC_Tumor vasculature 2", "RCC_Tumor vasculature 3", "RCC_Tumor vasculature 4"],
"Stromal" : ["MF_Adipo_CAR", "MF_Mature_MKs", "MF_OLCs", "MF_Osteoblasts", "RCC_Mesangial/vSMCs",
    "RCC_Myofibroblasts", "RCC_vSMCs"],
"Mast_cells" : ["MF_Mast_cells", "RCC_Mast cells"]}

In [ ]:
def build_dicts(df, clusters, opt = "all"):

        if opt == "all":
            sub_df = df[df["source"].isin(clusters) | df["target"].isin(clusters)]
        elif opt == "intra":
            sub_df = df[df["source"].isin(clusters) & df["target"].isin(clusters)]
            #sub_df = sub_df[sub_df["source"] != sub_df["target"]]
        elif opt == "inter":
            sub_df = df[df["source"].isin(clusters) | df["target"].isin(clusters)]
            sub_df = sub_df[~((sub_df["source"].isin(clusters)) & (sub_df["target"].isin(clusters)))]
        #print(sub_df)

        source_dict = {cl: [] for cl in clusters}
        target_dict = {cl: [] for cl in clusters}
        
        for _, row in sub_df.iterrows():
            if row["source"] in source_dict:
                source_dict[row["source"]].append(row["lr_pair"])
            if row["target"] in target_dict:
                target_dict[row["target"]].append(row["lr_pair"])
        return source_dict, target_dict

def threshold_intersection(sets_list, threshold):
                """Return elements appearing in at least threshold fraction of sets."""
                n_sets = len(sets_list)
                min_count = max(1, int(threshold * n_sets))  # at least 1
                counter = collections.Counter()
                for s in sets_list:
                    counter.update(s)
                return {elem for elem, count in counter.items() if count >= min_count}

    
def build_dicts_associated_gene(df, clusters, opt = "all"):

        if opt == "all":
            sub_df = df[df["source"].isin(clusters) | df["target"].isin(clusters)]
        elif opt == "intra":
            sub_df = df[df["source"].isin(clusters) & df["target"].isin(clusters)]
        elif opt == "inter":
            sub_df = df[df["source"].isin(clusters) | df["target"].isin(clusters)]
            sub_df = sub_df[~sub_df["source"].isin(clusters) & sub_df["target"].isin(clusters)]
        source_dict = {cl: [] for cl in clusters}
        target_dict = {cl: [] for cl in clusters}
        
        for _, row in sub_df.iterrows():
            if row["source"] in source_dict:
                source_dict[row["source"]].append(row["gene_A"])
            if row["target"] in target_dict:
                target_dict[row["target"]].append(row["gene_B"])
        return source_dict, target_dict

In [ ]:
#1a,b,c(set) 2a,b,c(threshold) 3a,b,c(unique)  get all interactions
#replace c with LR-L(downstream)
#d is the same as c but is returning source and target genes together (fix that receptor complexes still need to be split)
#add empty lists so that singleton clusters dont repeat genes

def get_interactions_set(opt = "set", opt2 = "a"):
    all_interactions_list = []
    all_interactions_source_list = []
    all_interactions_target_list = []
    unique_per_cluster = []
    unique_per_cluster_source = []
    unique_per_cluster_target = []
    test_clust = []

    for i in cluster_dict: 
        tst_clust = []
        all_interactions = []
        all_interactions_source = []
        all_interactions_target = []
        
        #for clust in cluster_dict[i]:
        #    if any(clust in vals for vals in stable_partners.values()):
        #        tst_clust.append(clust)

        for clust in cluster_dict[i]:
            tst_clust.append(clust)
            
        print(tst_clust)
        tst_clust = list(map(lambda x:  x.removeprefix("RCC_").removeprefix("MF_"), tst_clust))

        if opt2 == "a" or opt2 == "b":
            clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust)
            clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust)

        if opt2 == "c":
            clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust)
            clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust)

            downstreamL_RCC, _ = build_dicts_associated_gene(RCC, tst_clust)
            downstreamL_MF, _ = build_dicts_associated_gene(MF, tst_clust)

        if opt2 == "d" or opt2 == "d2":
            clust_dict_source_RCC, clust_dict_target_RCC = build_dicts_associated_gene(RCC, tst_clust)
            clust_dict_source_MF, clust_dict_target_MF = build_dicts_associated_gene(MF, tst_clust)

        RCC_source_sets =  {cl: set(clust_dict_source_RCC[cl]) for cl in tst_clust if len(clust_dict_target_RCC[cl]) != 0}
        MF_source_sets = {cl: set(clust_dict_source_MF[cl]) for cl in tst_clust if len(clust_dict_target_MF[cl]) != 0}

        RCC_target_sets =  {cl: set(clust_dict_target_RCC[cl]) for cl in tst_clust if len(clust_dict_target_RCC[cl]) != 0}
        MF_target_sets = {cl: set(clust_dict_target_MF[cl]) for cl in tst_clust if len(clust_dict_target_MF[cl]) != 0}

        if opt == "set" or opt == "unique":

            if opt2 == "c":

                downstreamL_RCC = {cl: set(downstreamL_RCC[cl]) for cl in tst_clust if len(downstreamL_RCC[cl]) != 0}
                downstreamL_MF = {cl: set(downstreamL_MF[cl]) for cl in tst_clust if len(downstreamL_MF[cl]) != 0}
                #print(downstreamL_RCC)

                RCC_target_sets= {
                cl: RCC_target_sets.get(cl, set()) | downstreamL_RCC.get(cl, set())
                    for cl in downstreamL_RCC.keys() if len(downstreamL_RCC[cl]) != 0}
                #print(downstreamL_RCC)

                MF_target_sets = {
                cl: MF_target_sets.get(cl, set()) | downstreamL_MF.get(cl, set())
                for cl in downstreamL_MF.keys()  if len(downstreamL_MF[cl]) != 0}
                
                #RCC_target_sets = {k: set(v) for k, v in RCC_target_sets.items()}
                #MF_target_sets = {k: set(v) for k, v in MF_target_sets.items()}
                

            RCC_source_sets_interactions = []
            RCC_target_sets_interactions = []
            RCC_total_interactions = []
            if len(RCC_target_sets) >= 1: 
                RCC_source_sets_interactions =  set.intersection(*RCC_source_sets.values())
                RCC_target_sets_interactions =  set.intersection(*RCC_target_sets.values())
                RCC_total_interactions = RCC_source_sets_interactions.union(RCC_target_sets_interactions)
                #print(RCC_target_sets)
                #print(RCC_target_sets_interactions)

            MF_source_sets_interactions = []
            MF_target_sets_interactions = []
            MF_total_interactions = []
            if len(MF_target_sets) >= 1: 
                MF_source_sets_interactions =  set.intersection(*MF_source_sets.values())
                MF_target_sets_interactions =  set.intersection(*MF_target_sets.values())
                MF_total_interactions = MF_source_sets_interactions.union(MF_target_sets_interactions)

            if opt2 == "a" or opt2 =="d":
                if len(RCC_total_interactions) > 1 and len(MF_total_interactions) > 1: 
                    all_interactions = RCC_total_interactions.intersection(MF_total_interactions)
                else:
                    all_interactions = set()
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

            if opt2 == "b"  or opt2 == "d2":
                if len(MF_source_sets_interactions) > 1 or len(RCC_source_sets_interactions) > 1: 
                    all_interactions_source = RCC_source_sets_interactions.intersection(MF_source_sets_interactions)
                    all_interactions_target = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)
                else:
                    all_interactions = set()

                all_interactions_source_list.append(all_interactions_source)
                all_interactions_target_list.append(all_interactions_target)
                test_clust.append(tst_clust)
                
            if opt2 == "c":
                if len(MF_target_sets_interactions) > 1 and len(RCC_target_sets_interactions) > 1: 
                    all_interactions = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)  
                else:
                    all_interactions = set()
                
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

        ###########################################################################################################  
        if opt == "threshold":

            if opt2 == "c":

                downstreamL_RCC = {cl: set(downstreamL_RCC[cl]) for cl in tst_clust if len(downstreamL_RCC[cl]) != 0}
                downstreamL_MF = {cl: set(downstreamL_MF[cl]) for cl in tst_clust if len(downstreamL_MF[cl]) != 0}
                #print(downstreamL_RCC)

                RCC_target_sets= {
                cl: RCC_target_sets.get(cl, set()) | downstreamL_RCC.get(cl, set())
                    for cl in downstreamL_RCC.keys() if len(downstreamL_RCC[cl]) != 0}
                #print(downstreamL_RCC)

                MF_target_sets = {
                cl: MF_target_sets.get(cl, set()) | downstreamL_MF.get(cl, set())
                for cl in downstreamL_MF.keys()  if len(downstreamL_MF[cl]) != 0}
                
                #RCC_target_sets = {k: set(v) for k, v in RCC_target_sets.items()}
                #MF_target_sets = {k: set(v) for k, v in MF_target_sets.items()}
                
            RCC_source_sets_list = list(RCC_source_sets.values())
            MF_source_sets_list = list(MF_source_sets.values())
            RCC_target_sets_list = list(RCC_target_sets.values())
            MF_target_sets_list = list(MF_target_sets.values())

            threshold = 0.7  # 80%

            source_interactions_RCC = threshold_intersection(RCC_source_sets_list, threshold) if len(RCC_source_sets_list) > 0 else set()
            target_interactions_RCC = threshold_intersection(RCC_target_sets_list, threshold) if len(RCC_target_sets_list) > 0 else set()

            source_interactions_MF = threshold_intersection(MF_source_sets_list, threshold) if len(MF_source_sets_list) > 0 else set()
            target_interactions_MF = threshold_intersection(MF_target_sets_list, threshold) if len(MF_target_sets_list) > 0 else set()

            all_interactions_RCC = source_interactions_RCC.union(target_interactions_RCC) if len(source_interactions_RCC) > 0 else set()
            all_interactions_MF = source_interactions_MF.union(target_interactions_MF) if len(source_interactions_MF) > 0 else set()

            if opt2 == "a" or opt2 == "d":
                all_interactions = all_interactions_RCC.intersection(all_interactions_MF) if (len(all_interactions_MF) > 0 & len(all_interactions_RCC)) else set()
                #all_interactions = list(threshold_intersection([all_interactions_RCC, all_interactions_MF]) if len(all_interactions_MF) > 0 else set())
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)
        
            if opt2 == "b" or opt2 == "d2":
                if len(source_interactions_MF) > 1 or len(source_interactions_RCC) > 1: 
                    all_interactions_source = source_interactions_RCC.intersection(source_interactions_MF)
                    all_interactions_target = target_interactions_RCC.intersection(target_interactions_MF)
                else:
                    all_interactions = set()

                all_interactions_source_list.append(all_interactions_source)
                all_interactions_target_list.append(all_interactions_target)
                test_clust.append(tst_clust)

            if opt2 =="c":
                if len(target_interactions_MF) > 1 and len(target_interactions_RCC) > 1: 
                    all_interactions = target_interactions_RCC.intersection(target_interactions_MF)  
                else:
                    all_interactions = set()
                
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

     
    if opt == "unique":

            if opt2 == "a" or opt2 == "d" or opt2 == "c":
                for i, clustergenes in enumerate(all_interactions_list):
                    #print(len(cluster))
                    if len(clustergenes) > 0:
                        #print("clustergenes", clustergenes)
                        set_i = set(clustergenes)  # interactions in this cluster
                        #print("set_i", set_i)
                        others = set().union(*[all_interactions_list[j] for j in range(len(all_interactions_list)) if j != i])
                        unique = set_i - others  # LRs only in this cluster
                        #print("unique", unique)
                        unique_per_cluster.append(unique)
                    else:
                        unique_per_cluster.append(set())

            if opt2 == "b" or opt2 == "d2":
                 
                for i, cluster in enumerate(all_interactions_source_list):
                    set_i = set(cluster)  # interactions in this cluster
                    others = set().union(*[all_interactions_source_list[j] for j in range(len(all_interactions_source_list)) if j != i])
                    unique = set_i - others  # LRs only in this cluster
                    unique_per_cluster_source.append(unique)

                for i, cluster in enumerate(all_interactions_target_list):
                    set_i = set(cluster)  # interactions in this cluster
                    others = set().union(*[all_interactions_target_list[j] for j in range(len(all_interactions_target_list)) if j != i])
                    unique = set_i - others  # LRs only in this cluster
                    unique_per_cluster_target.append(unique)


    if opt2 == "a" or opt2 =="c" or opt2 == "d":
        if opt == "set" or opt == "threshold":        
            return all_interactions_list, test_clust
        if opt == "unique":
            return unique_per_cluster, test_clust
        
    if opt2 == "b" or opt2 == "d2":
        if opt == "set" or opt == "threshold":        
            return all_interactions_source_list, all_interactions_target_list, test_clust
        if opt == "unique":
            return unique_per_cluster_source, unique_per_cluster_target, test_clust
        
    

In [ ]:
#e == just clust1 -L-R- clust2
#c2 is s= clust1-L-R-clust2 and t= just clust-L?? 
#a is clust1 -L-R-clust2-L-R-clust3
def get_intra_inter(opt, opt2, opt3):
    all_interactions_list = []
    all_interactions_source_list = []
    all_interactions_target_list = []
    unique_per_cluster = []
    unique_per_cluster_source = []
    unique_per_cluster_target = []
    test_clust = []
    all_interactions_dict = {}

    RCC_source_dict = {}
    RCC_target_dict = {}
    MF_source_dict = {}
    MF_target_dict = {}
    
    for i in cluster_dict: 
        tst_clust = []
        all_interactions = []
        all_interactions_source = []
        all_interactions_target = []
        
        for clust in cluster_dict[i]:
            tst_clust.append(clust)
            
        print(tst_clust)
        tst_clust = list(map(lambda x:  x.removeprefix("RCC_").removeprefix("MF_"), tst_clust))
        
        if opt3 == "intra":
            if opt2 == "a" or opt2 =="a2" or opt2 == "b" or opt2 =="e":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust, opt = "intra")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust, opt = "intra")

            if opt2 == "c":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust, opt = "intra")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust, opt = "intra")
                
                downstreamL_RCC, _ = build_dicts_associated_gene(RCC, tst_clust, opt = "intra")
                downstreamL_MF, _ = build_dicts_associated_gene(MF, tst_clust, opt = "intra")

            if opt2 == "d":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts_associated_gene(RCC, tst_clust, opt = "intra")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts_associated_gene(MF, tst_clust, opt = "intra")

        elif opt3 == "inter":
            if opt2 == "a"  or opt2 =="a2" or opt2 == "b" or opt2 =="e":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust, opt = "inter")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust, opt = "inter")

            if opt2 == "c":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust, opt = "inter")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust, opt = "inter")

                downstreamL_RCC, _ = build_dicts_associated_gene(RCC, tst_clust, opt = "all")
                downstreamL_MF, _ = build_dicts_associated_gene(MF, tst_clust, opt = "all")

            if opt2 == "d":
                clust_dict_source_RCC, clust_dict_target_RCC = build_dicts_associated_gene(RCC, tst_clust, opt = "inter")
                clust_dict_source_MF, clust_dict_target_MF = build_dicts_associated_gene(MF, tst_clust, opt = "inter")
        #print("clust dict source RCC", clust_dict_source_RCC)
        RCC_source_sets =  {cl: set(clust_dict_source_RCC[cl]) for cl in tst_clust if len(clust_dict_target_RCC[cl]) != 0}
        MF_source_sets = {cl: set(clust_dict_source_MF[cl]) for cl in tst_clust if len(clust_dict_target_MF[cl]) != 0}
        #print("rcc source sets", RCC_source_sets)
        
        RCC_target_sets =  {cl: set(clust_dict_target_RCC[cl]) for cl in tst_clust if len(clust_dict_target_RCC[cl]) != 0}
        MF_target_sets = {cl: set(clust_dict_target_MF[cl]) for cl in tst_clust if len(clust_dict_target_MF[cl]) != 0}

        if opt == "set" or opt == "unique":

            if opt2 == "c":

                downstreamL_RCC = {cl: set(downstreamL_RCC[cl]) for cl in tst_clust if len(downstreamL_RCC[cl]) != 0}
                downstreamL_MF = {cl: set(downstreamL_MF[cl]) for cl in tst_clust if len(downstreamL_MF[cl]) != 0}
                #print(downstreamL_RCC)

                RCC_target_sets= {
                cl: RCC_target_sets.get(cl, set()) | downstreamL_RCC.get(cl, set())
                    for cl in downstreamL_RCC.keys() if len(downstreamL_RCC[cl]) != 0}
                #print(downstreamL_RCC)

                MF_target_sets = {
                cl: MF_target_sets.get(cl, set()) | downstreamL_MF.get(cl, set())
                for cl in downstreamL_MF.keys()  if len(downstreamL_MF[cl]) != 0}
                
                #RCC_target_sets = {k: set(v) for k, v in RCC_target_sets.items()}
                #MF_target_sets = {k: set(v) for k, v in MF_target_sets.items()}
                

            RCC_source_sets_interactions = []
            RCC_target_sets_interactions = []
            RCC_total_interactions = []
            if len(RCC_target_sets) >= 1: 
                RCC_source_sets_interactions =  set.intersection(*RCC_source_sets.values())
                RCC_target_sets_interactions =  set.intersection(*RCC_target_sets.values())
                RCC_total_interactions = RCC_source_sets_interactions.union(RCC_target_sets_interactions)
                #print(RCC_target_sets)
                #print("RCC_source_sets_interactions", RCC_source_sets_interactions)
                #print("RCC_target_sets_interactions", RCC_target_sets_interactions)
                

            MF_source_sets_interactions = []
            MF_target_sets_interactions = []
            MF_total_interactions = []
            if len(MF_target_sets) >= 1: 
                MF_source_sets_interactions =  set.intersection(*MF_source_sets.values())
                MF_target_sets_interactions =  set.intersection(*MF_target_sets.values())
                MF_total_interactions = MF_source_sets_interactions.union(MF_target_sets_interactions)
                #print("MF_source_sets_interactions", MF_source_sets_interactions)
                #print("MF_target_sets_interactions", MF_target_sets_interactions)
            if opt2 == "a" or opt2 =="d":
                if len(RCC_total_interactions) > 1 and len(MF_total_interactions) > 1: 
                    all_interactions = RCC_total_interactions.intersection(MF_total_interactions)
                else:
                    all_interactions = set()
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

            if opt2 == "b"  or opt2 == "d2":
                if len(MF_source_sets_interactions) > 1 or len(RCC_source_sets_interactions) > 1: 
                    all_interactions_source = RCC_source_sets_interactions.intersection(MF_source_sets_interactions)
                    all_interactions_target = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)
                else:
                    all_interactions = set()

                all_interactions_source_list.append(all_interactions_source)
                all_interactions_target_list.append(all_interactions_target)
                test_clust.append(tst_clust)
                
            if opt2 == "c":
                if len(MF_target_sets_interactions) > 1 and len(RCC_target_sets_interactions) > 1: 
                    all_interactions = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)  
                else:
                    all_interactions = set()
                
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

            if opt2 == "e":
                if len(MF_target_sets_interactions) > 1 and len(RCC_target_sets_interactions) > 1: 
                    all_interactions = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)  
                else:
                    all_interactions = set()
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)
            
            #print(RCC_total_interactions, MF_total_interactions)
            #print(RCC_target_sets_interactions, MF_target_sets_interactions)

            if opt2 =="a2":
                
                
                RCC_source_dict[i] = RCC_source_sets_interactions
                RCC_target_dict[i] = RCC_target_sets_interactions
                MF_source_dict[i] = MF_source_sets_interactions
                MF_target_dict[i] = MF_target_sets_interactions
                test_clust.append(tst_clust)
       

        ###########################################################################################################  
        if opt == "threshold":

            if opt2 == "c":

                downstreamL_RCC = {cl: set(downstreamL_RCC[cl]) for cl in tst_clust if len(downstreamL_RCC[cl]) != 0}
                downstreamL_MF = {cl: set(downstreamL_MF[cl]) for cl in tst_clust if len(downstreamL_MF[cl]) != 0}
                #print(downstreamL_RCC)

                RCC_target_sets= {
                cl: RCC_target_sets.get(cl, set()) | downstreamL_RCC.get(cl, set())
                    for cl in downstreamL_RCC.keys() if len(downstreamL_RCC[cl]) != 0}
                #print(downstreamL_RCC)
                #print("rcc target sets", RCC_target_sets)
                MF_target_sets = {
                cl: MF_target_sets.get(cl, set()) | downstreamL_MF.get(cl, set())
                for cl in downstreamL_MF.keys()  if len(downstreamL_MF[cl]) != 0}
                #print("MF target sets", MF_target_sets)
                #RCC_target_sets = {k: set(v) for k, v in RCC_target_sets.items()}
                #MF_target_sets = {k: set(v) for k, v in MF_target_sets.items()}
                
            RCC_source_sets_list = list(RCC_source_sets.values())
            MF_source_sets_list = list(MF_source_sets.values())
            RCC_target_sets_list = list(RCC_target_sets.values())
            MF_target_sets_list = list(MF_target_sets.values())

            threshold = 0.7  # %

            source_interactions_RCC = threshold_intersection(RCC_source_sets_list, threshold) if len(RCC_source_sets_list) > 0 else set()
            target_interactions_RCC = threshold_intersection(RCC_target_sets_list, threshold) if len(RCC_target_sets_list) > 0 else set()

            source_interactions_MF = threshold_intersection(MF_source_sets_list, threshold) if len(MF_source_sets_list) > 0 else set()
            target_interactions_MF = threshold_intersection(MF_target_sets_list, threshold) if len(MF_target_sets_list) > 0 else set()

            all_interactions_RCC = source_interactions_RCC.union(target_interactions_RCC) if len(source_interactions_RCC) > 0 else set()
            all_interactions_MF = source_interactions_MF.union(target_interactions_MF) if len(source_interactions_MF) > 0 else set()
            #print("target interactions RCC",target_interactions_RCC)
            #print("target interactions MF",target_interactions_MF)

            if opt2 == "a" or opt2 == "d":
                all_interactions = all_interactions_RCC.intersection(all_interactions_MF) if (len(all_interactions_MF) > 0 & len(all_interactions_RCC)) else set()
                #all_interactions = list(threshold_intersection([all_interactions_RCC, all_interactions_MF]) if len(all_interactions_MF) > 0 else set())
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)
        
            if opt2 == "b" or opt2 == "d2":
                if len(source_interactions_MF) > 1 or len(source_interactions_RCC) > 1: 
                    all_interactions_source = source_interactions_RCC.intersection(source_interactions_MF)
                    all_interactions_target = target_interactions_RCC.intersection(target_interactions_MF)
                else:
                    all_interactions = set()

                all_interactions_source_list.append(all_interactions_source)
                all_interactions_target_list.append(all_interactions_target)
                test_clust.append(tst_clust)

            if opt2 =="c":
                if len(target_interactions_MF) > 1 and len(target_interactions_RCC) > 1: 
                    all_interactions = target_interactions_RCC.intersection(target_interactions_MF)  
                else:
                    all_interactions = set()
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

     ######################
    if opt == "unique":

            if opt2 == "a" or opt2 == "d" or opt2 == "c":
                for i, clustergenes in enumerate(all_interactions_list):
                    #print(len(cluster))
                    if len(clustergenes) > 0:
                        #print("clustergenes", clustergenes)
                        set_i = set(clustergenes)  # interactions in this cluster
                        #print("set_i", set_i)
                        others = set().union(*[all_interactions_list[j] for j in range(len(all_interactions_list)) if j != i])
                        unique = set_i - others  # LRs only in this cluster
                        #print("unique", unique)
                        unique_per_cluster.append(unique)
                    else:
                        unique_per_cluster.append(set())

            if opt2 == "b" or opt2 == "d2":
                 
                for i, cluster in enumerate(all_interactions_source_list):
                    set_i = set(cluster)  # interactions in this cluster
                    others = set().union(*[all_interactions_source_list[j] for j in range(len(all_interactions_source_list)) if j != i])
                    unique = set_i - others  # LRs only in this cluster
                    unique_per_cluster_source.append(unique)

                for i, cluster in enumerate(all_interactions_target_list):
                    set_i = set(cluster)  # interactions in this cluster
                    others = set().union(*[all_interactions_target_list[j] for j in range(len(all_interactions_target_list)) if j != i])
                    unique = set_i - others  # LRs only in this cluster
                    unique_per_cluster_target.append(unique)

            if opt2 == "a2":
                
                source_dict_unique = {}
                target_dict_unique = {}
                
                

                if len(RCC_source_dict) > 1 and len(MF_source_dict) > 1:    
                    source_dict = {
                        cl: RCC_source_dict[cl].intersection(MF_source_dict.get(cl, set()))
                        for cl in RCC_source_dict.keys()
                    }

            
                if len(RCC_target_dict) > 1 and len(MF_target_dict) > 1:    
                    target_dict = {
                        cl: RCC_target_dict[cl].intersection(MF_target_dict.get(cl, set()))
                        for cl in RCC_target_dict.keys()
                    }

                

                for cl, genes in  source_dict.items():
                    set_i = set(genes)  # interactions in this cluster
                    #print(f"set {cl}", set_i)  
                    others = set().union(*[source_dict[j] for j in source_dict if j != cl])
                    #print("j", [RCC_source_sets[j] for j in RCC_source_sets.keys() if j != cl])
                    #print("others", others)
                    unique = set_i - others  # LRs only in this cluster
                    #print("unique", unique)
                    source_dict_unique[cl] = unique

            
                for cl, genes in  target_dict.items():
                    set_i = set(genes)  # interactions in this cluster
                    #print(f"set {cl}", set_i)  
                    others = set().union(*[target_dict[j] for j in target_dict if j != cl])
                    unique = set_i - others  # LRs only in this cluster
                    #print("unique", unique)
                    target_dict_unique[cl] = unique

                #print(source_dict)
                #print(target_dict)

                for cl, val in source_dict_unique.items():
                    source_dict_unique[cl] = {item.split("_", 1)[0] for item in source_dict_unique[cl]}

                for cl in source_dict_unique.keys():
                    all_interactions_list.append(source_dict_unique[cl].union(target_dict_unique[cl]))
                            
                #all_interactions_list = list(all_interactions_dict.values())

                print("all interactions", all_interactions_list)

    if opt2 == "a" or opt2 =="c" or opt2 == "d" or opt2 == "e":
        if opt == "set" or opt == "threshold":     
            print("all interactions", all_interactions_list)   
            return all_interactions_list, test_clust
        if opt == "unique":
            print("all interactions", all_interactions_list)
            return unique_per_cluster, test_clust
        
    if opt2 == "a2":
        if opt == "unique":
            return all_interactions_list, test_clust

    if opt2 == "b" or opt2 == "d2":
        if opt == "set" or opt == "threshold":        
            return all_interactions_source_list, all_interactions_target_list, test_clust
        if opt == "unique":
            return unique_per_cluster_source, unique_per_cluster_target, test_clust
    

In [ ]:
#ff, clusters_cleaned = get_interactions_set(opt ="threshold", opt2="c")


IFs: keratin (epithelial cells), vimentin (mesenchymal cells), desmin (muscle cells)?
rcc: VHL gene inactivated; PAX8 expression, CD10

In [ ]:
def get_GO_terms(opt = "filter"):
    from goatools.obo_parser import GODag

    go = GODag("go-basic.obo")

    anchors = {
        "Monocytes_Lymphocytes": [
            "GO:0002376", # immune system process
            "GO:0002520", # immune system development
            "GO:0006955", # immune response
            "GO:0030097", # hemopoiesis
            "GO:0002764", # immune response-regulating signaling pathway
            "GO:0002250", # adaptive immune response
            "GO:0006665" #sphingolipid  metabolic process
        ],
        "Tumor_Prog": [
            "GO:0007049", # cell cycle
            "GO:0008283", # cell proliferation
            "GO:0030154", # cell differentiation
            "GO:0001837", # epithelial to mesenchymal transition
            "GO:0001525", # angiogenesis
            "GO:0001666", # response to hypoxia,
            "GO:0002841", #regulation of T cell mediated immune response to tumor cell
            "GO:0002347", #response to tumor cell
            "GO:0002357", # defense response to tumor cell
            "GO:0071228", #cellular response to tumor cell,
            "GO:0008219", #cell death
            "GO:0006914", #autophagy
            "GO:0006281", #DNA repair
            "GO:0007165",  #signal transduction
            "GO:0002248", #connective tissue replacement involved in inflammatory response wound healing
            "GO:0042060", #wound healing
            "GO:1903747", #positive regulation of fibronectin production
            "GO:0006986", #response to unfolded protein
            "GO:0036503" #ERAD pathway
        ],
        "Kidney": [
            "GO:0072001", # renal system development
            "GO:0001822", # kidney development
            "GO:0072073", # kidney epithelial cell differentiation
            "GO:0072078", # nephron development
        ],
        "Endothelial": [
            "GO:0001944", # vasculature development
            "GO:0001525", # angiogenesis
            "GO:0001568", # blood vessel development
            "GO:0003018", # vascular process in circulatory system
        ],
        "Stromal_Bone": [
            "GO:0001501", # skeletal system development
            "GO:0001649", # osteoblast differentiation
            "GO:0030198", # extracellular matrix organization
            "GO:0043062", # extracellular structure organization
            "GO:0048659", #smooth muscle cell proliferation
            "GO:0003012", # muscle system process 
            "GO:0048762", # mesenchymal cell differentiation
            "GO:0007155", #cell adhesion
            "GO:0098645", #collagen network
            "GO:0005581", #collagen trimer
            "GO:0001503",#ossification
            "GO:0060612" #adipose tissue development
        ],
        "MFs": [
            "GO:0005126", # cytokine receptor binding
            "GO:0031723", #chemokine receptor binding: 
            "GO:0003823", #antigen binding
            "GO:0015036", #oxidoreductase activity
            "GO:0000155", #kinase activity
            "GO:0000121", #phosphatase activity
            "GO:0005520", #growth factor binding:
            "GO:0048018", #receptor ligand activity
            "GO:0000981", #DNA-binding transcription factor activity
            "GO:0005518", #collagen binding
            "GO:0046810", #extracellular matrix binding
            "GO:0034987", #immunoglobulin receptor binding
            "GO:0008201", #heparin binding:
            "GO:0005125", #cytokine activity
            "GO:0005260", #intracellularly ATP-gated chloride channel activity
            "GO:0005201", #extracellular matrix structural constituent
            "GO:0008191", #metalloendopeptidase activity
            "GO:0005546", #phosphatidylinositol-4,5-bisphosphate binding
            "GO:0004222", #cysteine-type endopeptidase activity	ECM degradation during remodeling
            "GO:0008237",	#metallopeptidase activity
            "GO:0019961" #interferon binding
        ]
    }

    from goatools.associations import read_gaf


    # Load annotations (GAF file, e.g. human gene2go)
    #gene2go = read_gaf("goa_human.gaf")  # download from EBI/GOA

    # Count genes per GO term
    #go_counts = {go_id: len(genes) for go_id, genes in gene2go.items()}

    #print("GO:0001944 has", go_counts.index("0001944"))

    def get_children(go_ids):
        terms = set()
        terms_children = set()
        for go_id in go_ids:
            if go_id in go:
                terms.add(go_id)
                terms.update(go[go_id].get_all_children())
                terms_children.update(go[go_id].get_all_children())
        return terms

    # collect all terms across categories
    allowed_terms = set()
    for category, go_ids in anchors.items():
        allowed_terms.update(get_children(go_ids))
    print(len(allowed_terms))

    if opt == "filter":
        def load_annotations_from_gaf(path):
            """Load a GAF file (can be gzipped). Returns:
            - gene_to_go: dict[gname] -> set(GO IDs)
            - go_to_genes: dict[go] -> set(gname)
            Uses DB_Object_Symbol (col 3, zero-based index 2) as gene/key and GO_ID (col 5, index 4).
            """
            df = pd.read_csv(path, sep='\t', comment='!', header=None, dtype=str)
            # ensure at least 5 columns
            df = df.loc[:, [2,4]].dropna()
            df.columns = ['gene','go']
            gene_to_go = df.groupby('gene')['go'].agg(lambda s: set(s)).to_dict()
            go_to_genes = df.groupby('go')['gene'].agg(lambda s: set(s)).to_dict()
            return gene_to_go, go_to_genes


        def count_term_size(go_to_genes):
            """Global term size (unique genes per GO)"""
            return {go: len(genes) for go, genes in go_to_genes.items()}

        gene_to_go, go_to_genes = load_annotations_from_gaf("goa_human.gaf")

        term_sizes = count_term_size(go_to_genes)
        filtered_terms = {go for go, n in term_sizes.items() if n <=400}

        allowed_terms = allowed_terms.intersection(go_to_genes.keys())
        print(len(allowed_terms))

        filtered_out_terms = ['cardiac muscle cell differentiation', 'lung vasculature development',
                      "antigen processing and presentation", "cell differentiation", "positive regulation of gene expression", 
                      #"positive regulation of cell population proliferation",
                      "inflammatory response", "signal transduction", 
                      "innate immune response", "immune response", "immune system process","adaptive immune response", 
                      "cell population proliferation", "lung epithelial cell differentiation", "microglial cell activation", 
                      "astrocyte activation involved in immune response", "microglia development", "complement activation", 
                      "complement activation, classical pathway", "complement activation, lectin pathway", 
                      "complement activation, alternative pathway", "complement activation, GZMK pathway", #
                      "complement receptor mediated signaling pathway", "neurogenesis", "microglial cell activation involved in immune response", 
                      'immune effector process', 'cardiac muscle cell myoblast differentiation', 'kidney development', 'ureteric bud development',
                      'retina vasculature development in camera-type eye', 'glial cell proliferation', 'inner ear auditory receptor cell differentiation',
                      'neuroendocrine cell differentiation', 'aorta development', #'glial cell differentiation', 
                      'coronary vasculature development', "neuron differentiation", "dopaminergic neuron differentiation", 
                      'endocardial cushion to mesenchymal transition', 'epithelial to mesenchymal transition involved in endocardial cushion formation', 
                      'angiogenesis involved in coronary vascular morphogenesis', 'cerebellar granule cell precursor proliferation', 
                      "neural precursor cell proliferation", 'negative regulation of blood-brain barrier permeability',
                      'Bergmann glial cell differentiation', 'type II pneumocyte differentiation', 'type I pneumocyte differentiation', "microglial cell differentiation", 
                      'Cajal-Retzius cell differentiation', "neuroblast proliferation", "T cell differentiation in thymus", "odontoblast differentiation",
                      "cardiac epithelial to mesenchymal transition", "cardiac muscle contraction", "central nervous system vasculogenesis", 
                      "calcineurin-mediated signaling", "calcium-mediated signaling", "apoptotic process", 'Sertoli cell differentiation', "cell adhesion",
                      "G protein-coupled receptor signaling pathway"]

    return allowed_terms, filtered_out_terms


In [ ]:
def print_go_mapping_stats(res, all_genes):
    """
    Prints summary statistics on how many genes were successfully mapped to GO terms.

    Parameters
    ----------
    res : list
        Result of mygene.MyGeneInfo().querymany() call.
    all_genes : list
        List of all input genes (symbols) that were queried.
    """
    mapped_genes = set()
    unmapped_genes = set(all_genes)

    for r in res:
        if "go" in r and r.get("entrezgene") is not None:
            symbol = r.get("symbol", r.get("query"))
            mapped_genes.add(symbol)
            if symbol in unmapped_genes:
                unmapped_genes.remove(symbol)

    print("GO Mapping")
    print(f"Total genes: {len(all_genes)}")
    print(f"Number of mapped genes: {len(mapped_genes)}")
    print(f"Number of unmapped genes: {len(unmapped_genes)}")
    print(f"Percentage mapped: {len(mapped_genes)/len(all_genes)*100:.2f}%")
    #print("\nMapped genes:", list(mapped_genes))
    print("Unmapped genes:", list(unmapped_genes))

    return mapped_genes, unmapped_genes


In [ ]:
#ripped from pycrosstalker
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.colors as pc
import plotly.graph_objects as go
from plotnine import *
from adjustText import adjust_text
from gprofiler import GProfiler
from sankeyflow import Sankey
import json

def plot_sankey_half(lrobj_tbl, target = None, ligand_cluster = None, receptor_cluster = None, plt_name = None, threshold = 50, tfflag = True):
    """
    This function selected genes sankey plot

     Parameters
    ----------
    lrobj_tbl :
        LRobject table with all data

    target :
        gene

    ligand_cluster :
        Ligand Clusters

    receptor_cluster :
        Receptor Clusters

    plt_name :
        plot title

    threshold :
        top_n n value
    
    Returns
    -------
    Python default plot

    """

    lrobj_tbl = lrobj_tbl[(lrobj_tbl['type_gene_A'] == "Ligand") & (lrobj_tbl['type_gene_B'] == "Receptor")]

    if target is not None:
        if len(target.split('|')) > 1:
            target_type = str(target.split('|')[1])
            if target_type == 'R':
                if lrobj_tbl['gene_B'].str.contains('\\|').any():
                    pass
                else:
                    target = target.split('|')[0]
                data = lrobj_tbl[lrobj_tbl['gene_B'] == target]
            elif target_type == 'L':
                if lrobj_tbl['gene_A'].str.contains('\\|').any():
                    pass
                else:
                    target = target.split('|')[0]
                data = lrobj_tbl[lrobj_tbl['gene_A'] == target]
        else:
            data = lrobj_tbl[lrobj_tbl['allpair'].str.contains(target)]
    else:
        data = lrobj_tbl

    
    if ligand_cluster is not None:
        data = data[data['source'].isin(ligand_cluster)]
    
    if receptor_cluster is not None:
        data = data[data['target'].isin(receptor_cluster)]


    color_palette = ['#00BFC4', '#FF3E3E']

    
    if len(data) >= 1:
        cat_cols = ['source', 'gene_A']
        value_cols = 'LRScore'
        data = data.loc[data['LRScore'].abs().nlargest(min(len(data), threshold)).index]
        title = plt_name

        gen_sankey(data, cat_cols, value_cols, title)
    
    else:
        print(f"Gene->{target} Not Found")
    

def gen_sankey(df, cat_cols=[], value_cols='', title='Sankey Diagram'):
    """
    Helper function to the function plot_sankey()

     Parameters
    ----------
    df :
        Dataframe

    cat_cols :
        Columns interested in the sankey plot

    value_cols :
        Sankey plot generated using connections based on this value_cols

    title :
        Title of Sankey plot
    
    Returns
    -------
    Nothing (plots Sankey plot)

    """

    # df['source'] += 'S'
    df['target'] += ' '
    
    labelList = []
    for catCol in cat_cols:
        labelListTemp =  list((df[catCol].values))
        labelList = labelList + labelListTemp    
        
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
    
    vmin = sourceTargetDf['count'].min()
    vmax = sourceTargetDf['count'].max()
    limit = max(abs(vmin), abs(vmax))
    vcenter = 0
    norm = mcolors.TwoSlopeNorm(vmin=-limit, vcenter=vcenter, vmax=limit)
    # norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    cmap = plt.get_cmap('RdBu_r')
    sourceTargetDf['hex_color'] = sourceTargetDf['count'].apply(lambda x: mcolors.to_hex(cmap(norm(x))))
    
    flows = []
    for i, row in sourceTargetDf.iterrows():
        flows.append((row['source'], row['target'], 1, {'color': row['hex_color']}))

    nodes = Sankey.infer_nodes(flows)
    nodes_new = []
    for level in nodes:
        level_new = []
        for node in level:
            node_new = node + [{'color' : 'black',
                                'label_pos':'center', 'label_opts': dict(fontsize=10, bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='white'))}]
            level_new.append(node_new)
        nodes_new.append(level_new)

    fig, ax = plt.subplots(figsize=(15, 10))
    s = Sankey(flows=flows,
               nodes=nodes_new,
               flow_color_mode_alpha=0.3,
               node_opts=dict(label_format='{label}'),
    )
    s.draw(ax=ax)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.01, shrink=0.5)
    cbar.set_label(value_cols, fontsize=12)

    ax.text(x=0.0, y=1.02, s="Source", fontsize=10)
    ax.text(x=1.0, y=1.02, s="Ligand", fontsize=10)

    ax.set_title(title)
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_GO(f):
    

    allowed_terms, filtered_out_terms = get_GO_terms()

    clusters = list(cluster_dict.values())

    all_interactions = f
    df_full = pd.DataFrame() 
    genes_bps_sep = {}
    top_terms_list = []
    top_terms_list_MF = []
    for i in range(len(all_interactions)):
        markers_tst = list(all_interactions[i])

        s = "_"
        clust_name = s.join(clusters[i]).replace("/", "_")
        print(clust_name)
        print(clusters[i])
        pre_genes = []
        for m in markers_tst:
            splitt = m.split("_")
            pre_genes.append(splitt)

        genes = []
        for xs in pre_genes:
            for x in xs:
                genes.append(x)
                
        genes = list(np.unique(genes))

        mg = mygene.MyGeneInfo()

        res = mg.querymany(
            genes,
            scopes="symbol",
            fields="entrezgene,ensembl.gene,go",
            species="human"
        )

        rows = []
        for r in res:
            geneid = r.get("entrezgene")
            genename = r.get("symbol", r["query"]) 
            if geneid is None or "go" not in r:
                continue
            gos = r["go"]
            if isinstance(gos, dict):
                for ns, terms in gos.items():  # ns = BP, MF, CC
                    if isinstance(terms, list):
                        for t in terms:
                            rows.append({"Gene": genename, "GeneID": geneid, "GOID": t["id"], "Namespace": ns, "GOName": t["term"]})
                    elif isinstance(terms, dict):
                        rows.append({"Gene": genename, "GeneID": geneid, "GOID": terms["id"], "Namespace": ns, "GOName": terms["term"]})


        df = pd.DataFrame(rows)
        if not df.empty:
            df = df[df["GOID"].isin(allowed_terms)]
            df = df[~df["GOName"].isin(filtered_out_terms)]
            #df = df[df["GOName"].str.contains("pathway")==False]
            df = df[df["GOName"].str.contains("cardiac")==False]
            df = df[df["GOName"].str.contains("lens")==False]
            #df = df[df["GOName"].str.contains("thymic")==False]
            df = df[df["GOName"].str.contains("neural")==False]
            df = df[df["GOName"].str.contains("neuron")==False]
            df = df[df["GOName"].str.contains("brain")==False]
            df = df[df["GOName"].str.contains("Langerhans")==False]
            genes_bps_tmp = df.groupby("GOName")["Gene"].apply(lambda x: list(set(x)))

        genes_bps_sep[i] = genes_bps_tmp
        df_full = pd.concat((df_full, df))

        if df.empty:
            top_terms_list.append(set())
            top_terms_list_MF.append(set())
            continue

        go_counts = df.groupby(["Namespace", "GOID", "GOName"]).count().rename(columns={"GeneID": "GeneCount"}).reset_index()

        print(pre_genes)
        unique_pairs = df[["Gene", "GOName"]].drop_duplicates()
        for gene, go in zip(unique_pairs["Gene"], unique_pairs["GOName"]):
            print(gene, "→", go)

        mapped_genes, unmapped_genes = print_go_mapping_stats(res, genes)
        
        for ns in ["BP", "MF", "CC"]:
        
            top_terms = go_counts[go_counts.Namespace==ns].sort_values("GeneCount", ascending=False)
            top_terms_plot = go_counts[go_counts.Namespace==ns].sort_values("GeneCount", ascending=False).head(30)
            
            plt.figure(figsize=(10,6))
            sns.barplot(
                data=top_terms_plot,
                y="GOName",
                x="GeneCount",
                palette="viridis"
            )
            if ns == "BP":
                top_terms_list.append(top_terms["GOName"])
            elif ns == "MF":
                top_terms_list_MF.append(top_terms["GOName"])

            plt.xlabel("Number of Genes")
            plt.ylabel(f"GO {ns} Terms")
            #plt.title(f"{clust_name}")
            plt.tight_layout()
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1a_common_LRs_cluster_all_LR_pairs_unchanged/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1b_common_LRs_cluster_separate_incoming_outgoing/incoming/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1b_common_LRs_cluster_separate_incoming_outgoing/outgoing/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform/incoming/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform/outgoing/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform/set/RBERVertex/{clust_name}_top_20_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/RBConfig_1_25/set/c/{clusters[i][0]}_top_30_BPs_GO.png")
            #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/endgame/CPM_0_31/unique/c/{clusters[i][0]}_top_30_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\RBConfig_1_25\\threshold\\c\\{clusters[i][0]}_top_30_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\RBConfig_1_25\\set\\c\\{clusters[i][0]}_top_30_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform\\set\\RBERVertex\\unique\\{clusters[i][0]}_top_20_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\res025\\unique\\{clusters[i][0]}_top_20_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\res031\\unique_c\\{clusters[i][0]}_top_20_BPs_GO.png")
            #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\res031\\unique_d\\{clusters[i][0]}_top_20_BPs_GO.png")
            plt.show()
    return df_full, top_terms_list, top_terms_list_MF


In [ ]:
def plot_unique_bp_sankeys(f, clusters_cleaned, df_full, top_terms_list, intra_inter, which_os = "linux", cluster_dict = cluster_dict, overview = None, experimental = None):    

    def flatten(xss):
        return [x for xs in xss for x in xs]
    
    all_interactions = f
    clusters = list(cluster_dict.values())
    #print(df_full)
    df_full_bp = df_full[df_full["Namespace"] == "BP"]
    df_full_bp = df_full_bp[["Gene", "GOName"]]
    
    #df_full_bp.pivot(index="GOName", columns="Gene")
    genes_bps = df_full_bp.groupby("GOName")["Gene"].apply(lambda x: list(set(x)))

    #get bp terms that only appear in one single cluster
    unique_per_cluster = []

    #the <2 not important here, just didnt delete bc i copied it from the unique genes function
    for i, cluster in enumerate(top_terms_list):
        if len(clusters[i]) < 1:
            unique_per_cluster.append([])
            continue
        set_i = set(cluster)  # interactions in this cluster
        others = set().union(*[top_terms_list[j]for j in range(len(top_terms_list)) if j != i])
        unique = set_i - others  # LRs only in this cluster
        unique_per_cluster.append(unique)

    for idx, uniq in enumerate(unique_per_cluster):
        cluster_key = list(cluster_dict.keys())[idx]
        print(f"{cluster_key}: {uniq}")

    genes_bps_sub = genes_bps[genes_bps.index.isin(flatten(unique_per_cluster))]


    #doesnt check if lr pair is in cluster if using methods 1c where genes are already split
    #können einfach einzeln da sein
    method= "d"

    interactions_in_bp = {}
    interaction_genes_only = {}


    for i, lr_sets in enumerate(all_interactions):
        if not (lr_sets):
            continue
        cluster_hits = []
        interaction_genes = []
        #print(unique_per_cluster[i])
        for j in unique_per_cluster[i]:
            
            bps_of_interest = [j]

            #print(bps_of_interest)
            genes_bps_sub = genes_bps[genes_bps.index.isin(bps_of_interest)]
            #print(genes_bps_sub)
            
            
            for interaction in lr_sets:
                # split lr into ligand and receptor
                if "_" in interaction:
                    genes = interaction.split("_")
                else:
                    genes = [interaction]
                #print(genes)
                for bp, genes_in_bp in genes_bps_sub.items():
                    if not isinstance(genes_in_bp, set):
                        genes_in_bp = set(genes_in_bp)

                    #print(genes_in_bp)
                    # are both l and r in this biological process
                    matching = [g for g in genes if g in genes_in_bp]
                    #matching = [(g, "ligand" if idx==0 else "receptor") 
                    #     for idx, g in enumerate(genes) if g in genes_in_bp]
                    #print(matching)
                    # if both ligand and receptor are in this BP then keep
                    if method == "d":
                        if len(matching) >= 1:  
                            cluster_hits.append((clusters[i], interaction, bp, matching))
                            interaction_genes.append(interaction)

                    else:
                        if len(matching) >= 2:  
                            cluster_hits.append((clusters[i], interaction, bp, matching))
                            interaction_genes.append(interaction)

        interactions_in_bp[i] = cluster_hits
        interaction_genes_only[i] = interaction_genes


    for i, val in interaction_genes_only.items():
        print(len(set(val)))

    import contextvars
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri, default_converter
    from rpy2.robjects.conversion import localconverter, set_conversion

    if which_os == "windows11":
        # Initialize rpy2 conversion system
        set_conversion(default_converter)

        # Copy current context to propagate conversion settings into threads
        ctx = contextvars.copy_context()

        def load_diff_table(path: str):
            def _read():
                readRDS = ro.r['readRDS']
                df = readRDS(path)
                tables = df.slots['tables']

                diff_table = tables[2]
                with localconverter(default_converter + pandas2ri.converter):
                    return ro.conversion.rpy2py(diff_table)
            # Run inside captured context
            return ctx.run(_read)


    # Choose the correct file paths
    if which_os == "linux":

        readRDS = ro.r['readRDS']
        df_RCC = readRDS("/home/larissa/Documents/Masterarbeit/RCC_results/crosstalker/no_subset_glom_removed/LR_data_final.Rds")
        df_MF = readRDS("/home/larissa/Documents/Masterarbeit/MF_results/crosstalker/all_celltypes/pval0_05/LR_data_final.Rds")

        
        tables_RCC = df_RCC.slots["tables"]
        diff_table_RCC = tables_RCC[2]
        RCC_diff_df = pandas2ri.rpy2py(diff_table_RCC)

        tables_MF = df_MF.slots["tables"]
        diff_table_MF = tables_MF[2]
        MF_diff_df = pandas2ri.rpy2py(diff_table_MF)

    elif which_os == "windows":
        readRDS = ro.r['readRDS']
        df_RCC = readRDS("/home/larissa/Documents/Masterarbeit/RCC_results/crosstalker/no_subset_glom_removed/LR_data_final.Rds")
        df_MF = readRDS("/home/larissa/Documents/Masterarbeit/MF_results/crosstalker/all_celltypes/pval0_05/LR_data_final.Rds")

        

        tables_RCC = df_RCC.slots["tables"]
        diff_table_RCC = tables_RCC[2]
        RCC_diff_df = pandas2ri.rpy2py(diff_table_RCC)

        tables_MF = df_MF.slots["tables"]

        diff_table_MF = tables_MF[2]
        MF_diff_df = pandas2ri.rpy2py(diff_table_MF)

    elif which_os == "windows11":
        df_RCC = "C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\RCC_results\\crosstalker\\final_filtering\\LR_data_final.Rds"
        df_MF = "C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\MF_results\\crosstalker\\final_filtering\\LR_data_final.Rds"


        RCC_diff_df = load_diff_table(df_RCC)
        MF_diff_df = load_diff_table(df_MF)


    RCC_diff_df["gene_A"] = RCC_diff_df["gene_A"].apply([lambda x: x.removesuffix("|L")])
    RCC_diff_df["gene_B"] = RCC_diff_df["gene_B"].apply([lambda x: x.removesuffix("|R")])
    RCC_diff_df["lr_pair"] = RCC_diff_df["gene_A"] + "_" + RCC_diff_df["gene_B"]
    RCC_diff_df = RCC_diff_df[(~RCC_diff_df["source"].isin(["Glomerular endothelium", "Podocytes", "OM Type A-ICs", "Type B-IC","Type A-ICs", "TAL of LOH"])) & (~RCC_diff_df["target"].isin(["Glomerular endothelium", "Podocytes", "OM Type A-ICs", "Type B-IC", "Type A-ICs", "TAL of LOH"]))]

    MF_diff_df["gene_A"] = MF_diff_df["gene_A"].apply([lambda x: x.removesuffix("|L")])
    MF_diff_df["gene_B"] = MF_diff_df["gene_B"].apply([lambda x: x.removesuffix("|R")])
    MF_diff_df["lr_pair"] = MF_diff_df["gene_A"] + "_" + MF_diff_df["gene_B"]
    MF_diff_df = MF_diff_df[(~MF_diff_df["source"].isin(["Fibroblasts"])) & (~MF_diff_df["target"].isin(["Fibroblasts"]))]

    #if not big_df.empty:
    #    interaction_genes_only = genes_only
    #print(interaction_genes_only)
    RCC_lrscores = pd.DataFrame()
    MF_lrscores = pd.DataFrame()
    RCC_df_list = []
    MF_df_list  = []

    for i in range(len(clusters_cleaned)):

        RCC_diff_df_filtered = RCC_diff_df[RCC_diff_df["source"].isin(clusters_cleaned[i]) | RCC_diff_df["target"].isin(clusters_cleaned[i])]
        MF_diff_df_filtered = MF_diff_df[MF_diff_df["source"].isin(clusters_cleaned[i]) | MF_diff_df["target"].isin(clusters_cleaned[i])]

        if intra_inter == "inter":

            if i in interaction_genes_only:
                #print("interaction_genes", interaction_genes_only)
                print(i)
                for gene in np.unique(interaction_genes_only[i]):
                    print(gene)
                    #RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["gene_B"] == gene)| (RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(RCC_diff_df_filtered_gene) > 0:
                        RCC_df_list.append(RCC_diff_df_filtered_gene)
                        RCC_lrscores = pd.concat((RCC_lrscores, RCC_diff_df_filtered_gene))

                    #MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["gene_A"] == gene) | (MF_diff_df_filtered["gene_B"] == gene) | (MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(MF_diff_df_filtered_gene) > 0:
                        MF_df_list.append(MF_diff_df_filtered_gene)
                        MF_lrscores = pd.concat((MF_lrscores, MF_diff_df_filtered_gene))

        elif intra_inter == "intra":
           
           if i in interaction_genes_only:
                #print("interaction_genes", interaction_genes_only)
                print(i)
                for gene in np.unique(interaction_genes_only[i]):
                    print(gene)
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["gene_B"] == gene)| (RCC_diff_df_filtered["lr_pair"] == gene)]
                    #RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(RCC_diff_df_filtered_gene) > 0:
                        RCC_df_list.append(RCC_diff_df_filtered_gene)
                        RCC_lrscores = pd.concat((RCC_lrscores, RCC_diff_df_filtered_gene))

                    #MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["gene_A"] == gene) | (MF_diff_df_filtered["gene_B"] == gene) | (MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(MF_diff_df_filtered_gene) > 0:
                        MF_df_list.append(MF_diff_df_filtered_gene)
                        MF_lrscores = pd.concat((MF_lrscores, MF_diff_df_filtered_gene))


    #print("df list", RCC_df_list)

    cluster_dict = {
        k: [v.removeprefix("RCC_").removeprefix("MF_") for v in vals]
        for k, vals in cluster_dict.items()
    }
    #mapping column to specified clusters
    mapping = {
        item: cluster
        for cluster, items in cluster_dict.items()
        for item in items
    }

    grouped_list_RCC = []
    grouped_list_MF = []

    for df in RCC_df_list:
        df["cluster_t"] = df["target"].map(mapping)
        df["cluster_s"] = df["source"].map(mapping)

        grouped = df.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
        "LRScore": "mean"
        #"source": "first",    # or " ".join, list, etc.
        #"lr_pair": "first"  # repeat for other cols
        })
        grouped_list_RCC.append(grouped)

    #print(grouped_list_RCC)
    final_df_RCC = pd.concat(grouped_list_RCC, ignore_index=True)

    for df in MF_df_list:
        df["cluster_t"] = df["target"].map(mapping)
        df["cluster_s"] = df["source"].map(mapping)

        #grouped = df.groupby("cluster")["LRScore"].mean().reset_index()
        grouped = df.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
        "LRScore": "mean"
        #"source": "first",    # or " ".join, list, etc.
        #"lr_pair": "first"  # repeat for other cols
        })
        grouped_list_MF.append(grouped)

    final_df_MF = pd.concat(grouped_list_MF, ignore_index=True)

    RCC_final = final_df_RCC.rename(columns={"LRScore": "LRScore_RCC"})
    MF_final = final_df_MF.rename(columns={"LRScore": "LRScore_MF"})

    merged = pd.merge(
        RCC_final,
        MF_final,
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"   # only keep common cluster-target combos
    )


    merged["direction_RCC"] = merged["LRScore_RCC"].apply(lambda x: "pos" if x > 0 else "neg")
    merged["direction_MF"] = merged["LRScore_MF"].apply(lambda x: "pos" if x > 0 else "neg")

    merged["same_direction"] = merged["direction_RCC"] == merged["direction_MF"]

    subsetmerged = merged[merged["same_direction"] == True].drop_duplicates()

    if intra_inter == "intra":
                subsetmerged = subsetmerged[(subsetmerged["cluster_s"] == subsetmerged["cluster_t"])]

    elif intra_inter == "inter":
                subsetmerged = subsetmerged[(subsetmerged["cluster_s"] != subsetmerged["cluster_t"])]

    trend = merged.groupby("lr_pair").agg(
        mean_RCC=("LRScore_RCC", "mean"),
        mean_MF=("LRScore_MF", "mean")
    ).reset_index()

    # Derive trend directions from the mean
    trend["trend_RCC"] = trend["mean_RCC"].apply(lambda x: "pos" if x > 0 else ("neg" if x < 0 else "zero"))
    trend["trend_MF"]  = trend["mean_MF"].apply(lambda x: "pos" if x > 0 else ("neg" if x < 0 else "zero"))

    # Check if directions match
    trend["same_overall_trend"] = trend["trend_RCC"] == trend["trend_MF"]
    trend_true = trend[trend["same_overall_trend"] == True]

    #mapped which cell types belong to which clusters and if they're the source or target cell type
    cluster_dict = {
        k: [v.removeprefix("RCC_").removeprefix("MF_") for v in vals]
        for k, vals in cluster_dict.items()
    }

    mapping = {
        item: cluster
        for cluster, items in cluster_dict.items()
        for item in items
    }

    grouped_list_RCC = []
    grouped_list_MF = []
    # Apply to each dataframe in RCC_df_list

    RCC_split = RCC_diff_df.copy()
    RCC_split["cluster_t"] = RCC_split["target"].map(mapping)
    RCC_split["cluster_s"] = RCC_split["source"].map(mapping)

    MF_split = MF_diff_df.copy()
    MF_split["cluster_t"] = MF_split["target"].map(mapping)
    MF_split["cluster_s"] = MF_split["source"].map(mapping)


    if intra_inter == "intra":
    #            RCC_split = RCC_split[(RCC_split["cluster_s"] == RCC_split["cluster_t"])]
    #            print(RCC_split)
    #            MF_split = MF_split[(MF_split["cluster_s"] == MF_split["cluster_t"])]  
    #
                RCC_split = RCC_split[(RCC_split["source"] != RCC_split["target"])]
                MF_split = MF_split[(MF_split["source"] != MF_split["target"])]  
    #elif intra_inter == "inter":
    #            RCC_split = RCC_split[(RCC_split["cluster_s"] != RCC_split["cluster_t"])]
    #            MF_split = MF_split[(MF_split["cluster_s"] != MF_split["cluster_t"])]  
                
    subsetmerged_RCC = RCC_split.merge(
        subsetmerged[["cluster_s", "cluster_t", "lr_pair"]],
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"
    )

    subsetmerged_MF = MF_split.merge(
        subsetmerged[["cluster_s", "cluster_t", "lr_pair"]],
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"
    )

    subsetmerged_RCC['gene_A'] = subsetmerged_RCC['gene_A'].astype(str) + '|L'
    subsetmerged_RCC['gene_B'] = subsetmerged_RCC['gene_B'].astype(str) + '|R'

    subsetmerged_MF['gene_A'] = subsetmerged_MF['gene_A'].astype(str) + '|L'
    subsetmerged_MF['gene_B'] = subsetmerged_MF['gene_B'].astype(str) + '|R'


    if overview ==  "yes":
        overview_RCC = subsetmerged_RCC
        overview_RCC["source"] = subsetmerged_RCC["cluster_s"]
        overview_RCC["target"] = subsetmerged_RCC["cluster_t"] 

        overview_MF = subsetmerged_MF
        overview_MF["source"] = subsetmerged_MF["cluster_s"]
        overview_MF["target"] = subsetmerged_MF["cluster_t"]      

    #if experimental == "yes":
                # Keep only Ligand-Receptor edges
    #            lrobj_tbl = subsetmerged_RCC
                # Incoming: source → ligand → receptor → target
    #            incoming = lrobj_tbl[['source', 'gene_A', 'gene_B', 'target', 'LRScore', "cluster_t"]].copy()

                # Outgoing: target → downstream ligand#
    #            outgoing = lrobj_tbl[['source', 'gene_A', 'LRScore']].copy()
    #            outgoing['downstream_l'] = outgoing['gene_A'] + "|DL"
    #            outgoing.rename(columns={'source': 'target', 'LRScore': 'LRScore_down'}, inplace=True)

                # Merge incoming with outgoing once
    #            merged_lr = incoming.merge(
    #                outgoing[['target', 'downstream_l', 'LRScore_down']],
    #                on='target',
    #                how='left'
    #            )

    for name, i in cluster_dict.items():
        #print(subset_RCC)
        #subsetmerged_RCC = subsetmerged_RCC[subsetmerged_RCC["lr_pair"].isin(interaction_genes_only[i])] 
        subsetmerged_RCC_s = subsetmerged_RCC[subsetmerged_RCC["source"].isin(i)] 
        subsetmerged_RCC_t = subsetmerged_RCC[subsetmerged_RCC["target"].isin(i)]

        #subsetmerged_MF = subsetmerged_MF[subsetmerged_MF["lr_pair"].isin(interaction_genes_only[i])] 
        subsetmerged_MF_s = subsetmerged_MF[subsetmerged_MF["source"].isin(i)]
        subsetmerged_MF_t = subsetmerged_MF[subsetmerged_MF["target"].isin(i)]

        if overview == "yes":
            
            subsetmerged_RCC_s = overview_RCC[overview_RCC["source"] == name] 
            subsetmerged_RCC_t = overview_RCC[overview_RCC["target"] == name]
        
            subsetmerged_MF_s = overview_MF[overview_MF["source"] == name]
            subsetmerged_MF_t = overview_MF[overview_MF["target"] == name]
             


        if intra_inter == "intra":
            print(name)
            print("RCC")
            pycrosstalker.pl.plot_sankey(subsetmerged_RCC_s,threshold = 50, plt_name = None)
            print("MF")
            pycrosstalker.pl.plot_sankey(subsetmerged_MF_s, threshold=50, plt_name =None)

        elif intra_inter == "inter":
            if experimental == "yes":
       
                #df_cluster = merged_lr[merged_lr['cluster_t'] == name]
                print(name)
                print("RCC")
                plot_sankey(subsetmerged_RCC, target_cluster = name, threshold =50, plt_name = None)
                #plot_sankey(subsetmerged_RCC_t,threshold = 100, plt_name = f"RCC  {name}")
                #print("MF")
                #plot_sankey(subsetmerged_MF_s, threshold=100, plt_name = f"MF {name}")
                #plot_sankey(subsetmerged_MF_t, threshold=100, plt_name = f"MF {name}")
            else:
                print(name)
                print("RCC")
                plot_sankey_half(subsetmerged_RCC_s, threshold =50, plt_name = None)
                pycrosstalker.pl.plot_sankey(subsetmerged_RCC_t,threshold = 50, plt_name =None)
                print("MF")
                plot_sankey_half(subsetmerged_MF_s, threshold=50, plt_name = None)
                pycrosstalker.pl.plot_sankey(subsetmerged_MF_t, threshold=50, plt_name =None)


    return subsetmerged, subsetmerged_RCC, subsetmerged_MF

In [ ]:
def plot_unique_mf_sankeys(f, clusters_cleaned, df_full, top_terms_list_MF, intra_inter, which_os = "linux", cluster_dict = cluster_dict, overview = None, experimental = None):    

    def flatten(xss):
        return [x for xs in xss for x in xs]
    
    all_interactions = f
    clusters = list(cluster_dict.values())
    df_full_mf = df_full[df_full["Namespace"] == "MF"]
    df_full_mf = df_full_mf[["Gene", "GOName"]]
    #print(df_full_mf)
    #print(top_terms_list_MF)
    #df_full_bp.pivot(index="GOName", columns="Gene")
    genes_mfs = df_full_mf.groupby("GOName")["Gene"].apply(lambda x: list(set(x)))

    #get bp terms that only appear in one single cluster
    unique_per_cluster = []

    #the <2 not important here, just didnt delete bc i copied it from the unique genes function
    for i, cluster in enumerate(top_terms_list_MF):
        if len(clusters[i]) < 1:
            #unique_per_cluster.append(set())
            continue
        set_i = set(cluster)  # interactions in this cluster
        others = set().union(*[top_terms_list_MF[j]for j in range(len(top_terms_list_MF)) if j != i])
        unique = set_i - others  # LRs only in this cluster
        unique_per_cluster.append(unique)


    for idx, uniq in enumerate(unique_per_cluster):
        cluster_key = list(cluster_dict.keys())[idx]
        print(f"{cluster_key}: {uniq}")

    genes_mfs_sub = genes_mfs[genes_mfs.index.isin(flatten(unique_per_cluster))]


    #doesnt check if lr pair is in cluster if using methods 1c where genes are already split
    #können einfach einzeln da sein
    method= "d"

    interactions_in_mf = {}
    interaction_genes_only_mf = {}


    for i, lr_sets in enumerate(all_interactions):
        if not (lr_sets):
            continue
        cluster_hits = []
        interaction_genes = []
        #print(unique_per_cluster[i])
        for j in unique_per_cluster[i]:
            
            mfs_of_interest = [j]

            #print(bps_of_interest)
            genes_mfs_sub = genes_mfs[genes_mfs.index.isin(mfs_of_interest)]
            #print(genes_bps_sub)
            
            
            for interaction in lr_sets:
                # split lr into ligand and receptor
                if "_" in interaction:
                    genes = interaction.split("_")
                else:
                    genes = [interaction]
                #print(genes)
                for mf, genes_in_mf in genes_mfs_sub.items():
                    if not isinstance(genes_in_mf, set):
                        genes_in_mf = set(genes_in_mf)

                    #print(genes_in_bp)
                    # are both l and r in this biological process
                    matching = [g for g in genes if g in genes_in_mf]
                    #matching = [(g, "ligand" if idx==0 else "receptor") 
                    #     for idx, g in enumerate(genes) if g in genes_in_bp]
                    #print(matching)
                    # if both ligand and receptor are in this BP then keep
                    if method == "d":
                        if len(matching) >= 1:  
                            cluster_hits.append((clusters[i], interaction, mf, matching))
                            interaction_genes.append(interaction)


                    else:
                        if len(matching) >= 2:  
                            cluster_hits.append((clusters[i], interaction, mf, matching))
                            interaction_genes.append(interaction)

        interactions_in_mf[i] = cluster_hits
        interaction_genes_only_mf[i] = interaction_genes

        
    print("inter action genes", interaction_genes_only_mf)
    for i, val in interaction_genes_only_mf.items():
        print(len(set(val)))

    import contextvars
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri, default_converter
    from rpy2.robjects.conversion import localconverter, set_conversion

    if which_os == "windows11":
        # Initialize rpy2 conversion system
        set_conversion(default_converter)

        # Copy current context to propagate conversion settings into threads
        ctx = contextvars.copy_context()

        def load_diff_table(path: str):
            def _read():
                readRDS = ro.r['readRDS']
                df = readRDS(path)
                tables = df.slots['tables']
                diff_table = tables[2]
                with localconverter(default_converter + pandas2ri.converter):
                    return ro.conversion.rpy2py(diff_table)
            # Run inside captured context
            return ctx.run(_read)


    # Choose the correct file paths
    if which_os == "linux":

        readRDS = ro.r['readRDS']
        df_RCC = readRDS("/home/larissa/Documents/Masterarbeit/RCC_results/crosstalker/no_subset_glom_removed/LR_data_final.Rds")
        df_MF = readRDS("/home/larissa/Documents/Masterarbeit/MF_results/crosstalker/all_celltypes/pval0_05/LR_data_final.Rds")

        
        tables_RCC = df_RCC.slots["tables"]
        diff_table_RCC = tables_RCC[2]
        RCC_diff_df = pandas2ri.rpy2py(diff_table_RCC)

        tables_MF = df_MF.slots["tables"]
        diff_table_MF = tables_MF[2]
        MF_diff_df = pandas2ri.rpy2py(diff_table_MF)

    elif which_os == "windows":
        readRDS = ro.r['readRDS']
        df_RCC = readRDS("/home/larissa/Documents/Masterarbeit/RCC_results/crosstalker/no_subset_glom_removed/LR_data_final.Rds")
        df_MF = readRDS("/home/larissa/Documents/Masterarbeit/MF_results/crosstalker/all_celltypes/pval0_05/LR_data_final.Rds")

        

        tables_RCC = df_RCC.slots["tables"]
        diff_table_RCC = tables_RCC[2]
        RCC_diff_df = pandas2ri.rpy2py(diff_table_RCC)

        tables_MF = df_MF.slots["tables"]
        diff_table_MF = tables_MF[2]
        MF_diff_df = pandas2ri.rpy2py(diff_table_MF)

    elif which_os == "windows11":
        df_RCC = "C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\RCC_results\\crosstalker\\final_filtering\\LR_data_final.Rds"
        df_MF = "C:\\Users\\laris\\Documents\\Studium\\Masterarbeit\\MF_results\\crosstalker\\final_filtering\\LR_data_final.Rds"

        RCC_diff_df = load_diff_table(df_RCC)
        MF_diff_df = load_diff_table(df_MF)


    RCC_diff_df["gene_A"] = RCC_diff_df["gene_A"].apply([lambda x: x.removesuffix("|L")])
    RCC_diff_df["gene_B"] = RCC_diff_df["gene_B"].apply([lambda x: x.removesuffix("|R")])
    RCC_diff_df["lr_pair"] = RCC_diff_df["gene_A"] + "_" + RCC_diff_df["gene_B"]
    RCC_diff_df = RCC_diff_df[(~RCC_diff_df["source"].isin(["Glomerular endothelium", "Podocytes", "OM Type A-ICs", "Type B-IC","Type A-ICs", "TAL of LOH"])) & (~RCC_diff_df["target"].isin(["Glomerular endothelium", "Podocytes", "OM Type A-ICs", "Type B-IC", "Type A-ICs", "TAL of LOH"]))]

    MF_diff_df["gene_A"] = MF_diff_df["gene_A"].apply([lambda x: x.removesuffix("|L")])
    MF_diff_df["gene_B"] = MF_diff_df["gene_B"].apply([lambda x: x.removesuffix("|R")])
    MF_diff_df["lr_pair"] = MF_diff_df["gene_A"] + "_" + MF_diff_df["gene_B"]
    MF_diff_df = MF_diff_df[(~MF_diff_df["source"].isin(["Fibroblasts"])) & (~MF_diff_df["target"].isin(["Fibroblasts"]))]

    #if not big_df.empty:
    #    interaction_genes_only = genes_only
    #print(interaction_genes_only)
    RCC_lrscores = pd.DataFrame()
    MF_lrscores = pd.DataFrame()
    RCC_df_list = []
    MF_df_list  = []

    for i in range(len(clusters_cleaned)):

        RCC_diff_df_filtered = RCC_diff_df[RCC_diff_df["source"].isin(clusters_cleaned[i]) | RCC_diff_df["target"].isin(clusters_cleaned[i])]
        MF_diff_df_filtered = MF_diff_df[MF_diff_df["source"].isin(clusters_cleaned[i]) | MF_diff_df["target"].isin(clusters_cleaned[i])]

        if intra_inter == "inter":

            if i in interaction_genes_only_mf:
                print("interaction_genes", interaction_genes_only_mf)
                print(i)
                for gene in np.unique(interaction_genes_only_mf[i]):
                    print(gene)
                    f = clusters_cleaned.iloc(i)
                    #RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["gene_B"] == gene)| (RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(RCC_diff_df_filtered_gene) > 0:
                        RCC_df_list.append(RCC_diff_df_filtered_gene)
                        RCC_lrscores = pd.concat((RCC_lrscores, RCC_diff_df_filtered_gene))

                    #MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["gene_A"] == gene) | (MF_diff_df_filtered["gene_B"] == gene) | (MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(MF_diff_df_filtered_gene) > 0:
                        MF_df_list.append(MF_diff_df_filtered_gene)
                        MF_lrscores = pd.concat((MF_lrscores, MF_diff_df_filtered_gene))
        elif intra_inter == "intra":
           
           if i in interaction_genes_only_mf:
                print("interaction_genes", interaction_genes_only_mf)
                print(i)
                for gene in np.unique(interaction_genes_only_mf[i]):
                    print(gene)
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["gene_A"] == gene) | (RCC_diff_df_filtered["gene_B"] == gene)| (RCC_diff_df_filtered["lr_pair"] == gene)]
                    #RCC_diff_df_filtered_gene = RCC_diff_df_filtered[(RCC_diff_df_filtered["lr_pair"] == gene)]
                    RCC_diff_df_filtered_gene = RCC_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(RCC_diff_df_filtered_gene) > 0:
                        RCC_df_list.append(RCC_diff_df_filtered_gene)
                        RCC_lrscores = pd.concat((RCC_lrscores, RCC_diff_df_filtered_gene))

                    MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["gene_A"] == gene) | (MF_diff_df_filtered["gene_B"] == gene) | (MF_diff_df_filtered["lr_pair"] == gene)]
                    #MF_diff_df_filtered_gene = MF_diff_df_filtered[(MF_diff_df_filtered["lr_pair"] == gene)]
                    MF_diff_df_filtered_gene = MF_diff_df_filtered_gene.sort_values("LRScore", ascending=False)
                    if len(MF_diff_df_filtered_gene) > 0:
                        MF_df_list.append(MF_diff_df_filtered_gene)
                        MF_lrscores = pd.concat((MF_lrscores, MF_diff_df_filtered_gene))

    cluster_dict = {
        k: [v.removeprefix("RCC_").removeprefix("MF_") for v in vals]
        for k, vals in cluster_dict.items()
    }
    #mapping column to specified clusters
    mapping = {
        item: cluster
        for cluster, items in cluster_dict.items()
        for item in items
    }

    grouped_list_RCC = []
    grouped_list_MF = []

    for df in RCC_df_list:

        df["cluster_t"] = df["target"].map(mapping)
        df["cluster_s"] = df["source"].map(mapping)

        grouped = df.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
        "LRScore": "mean"
        #"source": "first",    # or " ".join, list, etc.
        #"lr_pair": "first"  # repeat for other cols
        })
        grouped_list_RCC.append(grouped)

    #print(grouped_list_RCC)
    final_df_RCC = pd.concat(grouped_list_RCC, ignore_index=True)

    for df in MF_df_list:
        df["cluster_t"] = df["target"].map(mapping)
        df["cluster_s"] = df["source"].map(mapping)

        #grouped = df.groupby("cluster")["LRScore"].mean().reset_index()
        grouped = df.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
        "LRScore": "mean"
        #"source": "first",    # or " ".join, list, etc.
        #"lr_pair": "first"  # repeat for other cols
        })
        grouped_list_MF.append(grouped)

    final_df_MF = pd.concat(grouped_list_MF, ignore_index=True)

    RCC_final = final_df_RCC.rename(columns={"LRScore": "LRScore_RCC"})
    MF_final = final_df_MF.rename(columns={"LRScore": "LRScore_MF"})

    merged = pd.merge(
        RCC_final,
        MF_final,
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"   # only keep common cluster-target combos
    )


    merged["direction_RCC"] = merged["LRScore_RCC"].apply(lambda x: "pos" if x > 0 else "neg")
    merged["direction_MF"] = merged["LRScore_MF"].apply(lambda x: "pos" if x > 0 else "neg")

    merged["same_direction"] = merged["direction_RCC"] == merged["direction_MF"]

    subsetmerged = merged[merged["same_direction"] == True].drop_duplicates()

    if intra_inter == "intra":
                subsetmerged = subsetmerged[(subsetmerged["cluster_s"] == subsetmerged["cluster_t"])]

    elif intra_inter == "inter":
                subsetmerged = subsetmerged[(subsetmerged["cluster_s"] != subsetmerged["cluster_t"])]

    trend = merged.groupby("lr_pair").agg(
        mean_RCC=("LRScore_RCC", "mean"),
        mean_MF=("LRScore_MF", "mean")
    ).reset_index()

    # Derive trend directions from the mean
    trend["trend_RCC"] = trend["mean_RCC"].apply(lambda x: "pos" if x > 0 else ("neg" if x < 0 else "zero"))
    trend["trend_MF"]  = trend["mean_MF"].apply(lambda x: "pos" if x > 0 else ("neg" if x < 0 else "zero"))

    # Check if directions match
    trend["same_overall_trend"] = trend["trend_RCC"] == trend["trend_MF"]
    trend_true = trend[trend["same_overall_trend"] == True]

    #mapped which cell types belong to which clusters and if they're the source or target cell type
    cluster_dict = {
        k: [v.removeprefix("RCC_").removeprefix("MF_") for v in vals]
        for k, vals in cluster_dict.items()
    }

    mapping = {
        item: cluster
        for cluster, items in cluster_dict.items()
        for item in items
    }

    grouped_list_RCC = []
    grouped_list_MF = []
    # Apply to each dataframe in RCC_df_list

    RCC_split = RCC_diff_df.copy()
    RCC_split["cluster_t"] = RCC_split["target"].map(mapping)
    RCC_split["cluster_s"] = RCC_split["source"].map(mapping)

    MF_split = MF_diff_df.copy()
    MF_split["cluster_t"] = MF_split["target"].map(mapping)
    MF_split["cluster_s"] = MF_split["source"].map(mapping)


    if intra_inter == "intra":
    #            RCC_split = RCC_split[(RCC_split["cluster_s"] == RCC_split["cluster_t"])]
    #            print(RCC_split)
    #            MF_split = MF_split[(MF_split["cluster_s"] == MF_split["cluster_t"])]  
    #
                RCC_split = RCC_split[(RCC_split["source"] != RCC_split["target"])]
                MF_split = MF_split[(MF_split["source"] != MF_split["target"])]  
    #elif intra_inter == "inter":
    #            RCC_split = RCC_split[(RCC_split["cluster_s"] != RCC_split["cluster_t"])]
    #            MF_split = MF_split[(MF_split["cluster_s"] != MF_split["cluster_t"])]  
                
    subsetmerged_RCC = RCC_split.merge(
        subsetmerged[["cluster_s", "cluster_t", "lr_pair"]],
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"
    )

    subsetmerged_MF = MF_split.merge(
        subsetmerged[["cluster_s", "cluster_t", "lr_pair"]],
        on=["cluster_s", "cluster_t", "lr_pair"],
        how="inner"
    )

    subsetmerged_RCC['gene_A'] = subsetmerged_RCC['gene_A'].astype(str) + '|L'
    subsetmerged_RCC['gene_B'] = subsetmerged_RCC['gene_B'].astype(str) + '|R'

    subsetmerged_MF['gene_A'] = subsetmerged_MF['gene_A'].astype(str) + '|L'
    subsetmerged_MF['gene_B'] = subsetmerged_MF['gene_B'].astype(str) + '|R'


    if overview ==  "yes":
        overview_RCC = subsetmerged_RCC
        overview_RCC["source"] = subsetmerged_RCC["cluster_s"]
        overview_RCC["target"] = subsetmerged_RCC["cluster_t"] 

        overview_MF = subsetmerged_MF
        overview_MF["source"] = subsetmerged_MF["cluster_s"]
        overview_MF["target"] = subsetmerged_MF["cluster_t"]      

    #if experimental == "yes":
                # Keep only Ligand-Receptor edges
    #            lrobj_tbl = subsetmerged_RCC
                # Incoming: source → ligand → receptor → target
    #            incoming = lrobj_tbl[['source', 'gene_A', 'gene_B', 'target', 'LRScore', "cluster_t"]].copy()

                # Outgoing: target → downstream ligand#
    #            outgoing = lrobj_tbl[['source', 'gene_A', 'LRScore']].copy()
    #            outgoing['downstream_l'] = outgoing['gene_A'] + "|DL"
    #            outgoing.rename(columns={'source': 'target', 'LRScore': 'LRScore_down'}, inplace=True)

                # Merge incoming with outgoing once
    #            merged_lr = incoming.merge(
    #                outgoing[['target', 'downstream_l', 'LRScore_down']],
    #                on='target',
    #                how='left'
    #            )

    for name, i in cluster_dict.items():
        #print(subset_RCC)
        subsetmerged_RCC_s = subsetmerged_RCC[subsetmerged_RCC["source"].isin(i)] 
        subsetmerged_RCC_t = subsetmerged_RCC[subsetmerged_RCC["target"].isin(i)]

        subsetmerged_MF_s = subsetmerged_MF[subsetmerged_MF["source"].isin(i)]
        subsetmerged_MF_t = subsetmerged_MF[subsetmerged_MF["target"].isin(i)]

        if overview == "yes":
            
            subsetmerged_RCC_s = overview_RCC[overview_RCC["source"] == name] 
            subsetmerged_RCC_t = overview_RCC[overview_RCC["target"] == name]
        
            subsetmerged_MF_s = overview_MF[overview_MF["source"] == name]
            subsetmerged_MF_t = overview_MF[overview_MF["target"] == name]
             


        if intra_inter == "intra":
            print("RCC")
            pycrosstalker.pl.plot_sankey(subsetmerged_RCC_s,threshold = 100, plt_name =None)
            print("MF")
            pycrosstalker.pl.plot_sankey(subsetmerged_MF_s, threshold=100, plt_name = None)

        elif intra_inter == "inter":
            if experimental == "yes":
       
                #df_cluster = merged_lr[merged_lr['cluster_t'] == name]
                print("RCC")
                plot_sankey(subsetmerged_RCC, target_cluster = name, threshold =50, plt_name = None)
                #plot_sankey(subsetmerged_RCC_t,threshold = 100, plt_name = f"RCC  {name}")
                #print("MF")
                #plot_sankey(subsetmerged_MF_s, threshold=100, plt_name = f"MF {name}")
                #plot_sankey(subsetmerged_MF_t, threshold=100, plt_name = f"MF {name}")
            else:
                print("RCC")
                pycrosstalker.pl.plot_sankey(subsetmerged_RCC_s, threshold =50, plt_name =None)
                pycrosstalker.pl.plot_sankey(subsetmerged_RCC_t,threshold = 50, plt_name = None)
                print("MF")
                pycrosstalker.pl.plot_sankey(subsetmerged_MF_s, threshold=50, plt_name = None)
                pycrosstalker.pl.plot_sankey(subsetmerged_MF_t, threshold=50, plt_name =None)


    return subsetmerged, subsetmerged_RCC, subsetmerged_MF   


In [ ]:
def GO_stuff(opt, opt2, opt3, which_os = "linux", overview = None, experimental = None):
    f, clusters_cleaned = get_intra_inter(opt = opt, opt2= opt2, opt3= opt3)
    df_full, top_terms_list, top_terms_list_MF = plot_GO(f)
    subsetmerged, subsetmerged_RCC, subsetmerged_MF = plot_unique_bp_sankeys(f, clusters_cleaned, df_full, top_terms_list, intra_inter = opt3, which_os = which_os, overview = overview,  experimental = experimental)
    subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs = plot_unique_mf_sankeys(f, clusters_cleaned, df_full, top_terms_list_MF, intra_inter = opt3, which_os = which_os, overview = overview,  experimental = experimental)
 
    return df_full, top_terms_list, subsetmerged, subsetmerged_RCC, subsetmerged_MF, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs

In [ ]:
df_full_intra, top_terms_list_intra, subsetmerged_intra, subsetmerged_RCC_intra, subsetmerged_MF_intra, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs = GO_stuff(opt = "unique", opt2="a", opt3= "intra", which_os = which_os, overview  = "False")


In [ ]:
subsetmerged_RCC_intra

In [ ]:
df_full_inter, top_terms_list_inter, subsetmerged_inter, subsetmerged_RCC_inter, subsetmerged_MF_inter, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs  = GO_stuff(opt="unique", opt2="a2", opt3= "inter", which_os = which_os, overview = "no", experimental = "fuck off")

In [ ]:
subsetmerged_RCC_inter

In [ ]:
subsetmerged_RCC_intra.to_latex()

In [ ]:
df_full_inter, top_terms_list_inter, subsetmerged_inter, subsetmerged_RCC_inter, subsetmerged_MF_inter, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs  = GO_stuff(opt="unique", opt2="a2", opt3= "inter", which_os = "windows11", overview = "yes", experimental = "fuck off")

In [ ]:
subsetmerged_RCC_inter

In [ ]:
subsetmerged_MF_inter


In [ ]:
#overview
df_full_inter, top_terms_list_inter, subsetmerged_inter, subsetmerged_RCC_inter, subsetmerged_MF_inter, subsetmerged_gomfs, subsetmerged_RCC_gomfs, subsetmerged_MF_gomfs = GO_stuff(opt="unique", opt2="a2", opt3= "inter", which_os = which_os, overview = "yes", experimental = "fuck off")

In [ ]:
#experimental
df_full_inter, top_terms_list_inter, subsetmerged_inter, subsetmerged_RCC_inter, subsetmerged_MF_inter = GO_stuff(opt="unique", opt2="a2", opt3= "inter", which_os = which_os, overview = False, experimental = "yes")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from sankeyflow import Sankey

def plot_sankey(df, plt_name="Sankey Plot",
                        cat_cols=['source','gene_A','gene_B','target','downstream_l'],
                        value_col='LRScore',
                        threshold=50):
    """
    Plot Sankey for a pre-filtered dataframe of a single target cluster.
    """
    # Drop rows with no target (shouldn’t happen but safe)
    df = df.dropna(subset=['target'])
    df = df.nlargest(threshold, value_col)
    # Build flows
    flows = []
    for i in range(len(cat_cols)-1):
        pairs = df[[cat_cols[i], cat_cols[i+1], value_col]].dropna()
        for _, row in pairs.iterrows():
            flows.append((row[cat_cols[i]], row[cat_cols[i+1]], row[value_col], {}))

    if not flows:
        print("No valid flows for Sankey.")
        return

    # Color normalization
    vmin, vmax = df[value_col].min(), df[value_col].max()
    limit = max(abs(vmin), abs(vmax))
    cmap = cm.get_cmap('RdBu_r')
    norm = mcolors.TwoSlopeNorm(vmin=-limit, vcenter=0, vmax=limit)

    # Apply colors to flows
    for i, (s, t, v, meta) in enumerate(flows):
        meta['color'] = mcolors.to_hex(cmap(norm(v)))
        meta['alpha'] = 0.5
        flows[i] = (s, t, 1, meta)  # uniform thickness

    # Infer nodes for proper layout
    nodes = Sankey.infer_nodes(flows)

    fig, ax = plt.subplots(figsize=(14, 8))
    s = Sankey(
        flows=flows,
        nodes=nodes,
        flow_color_mode_alpha=0.3,
        node_opts=dict(label_format='{label}', label_pos='center')
    )
    s.draw(ax=ax)

    # Colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.02, shrink=0.6)
    cbar.set_label(value_col, fontsize=12)

    # Stage labels
    stages = ["Source", "Ligand", "Receptor", "Target", "Downstream L"]
    for i, stage in enumerate(stages):
        ax.text(i*1.0, 1.02, stage, fontsize=10)

    ax.set_title(plt_name, fontsize=14)
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_sankey(lrobj_tbl, target=None, target_cluster=None,
                      ligand_cluster=None, receptor_cluster=None,
                      plt_name=None, threshold=50):
    """
    Robust Sankey builder that ensures downstream ligand nodes appear.
    Uses either 'cluster_t' or 'target' for cluster-level matching depending on what's present.
    """

    def ensure_list(x):
        if x is None:
            return None
        return [x] if isinstance(x, str) else x

    target_cluster = ensure_list(target_cluster)
    ligand_cluster = ensure_list(ligand_cluster)
    receptor_cluster = ensure_list(receptor_cluster)

    df = lrobj_tbl.copy()

    # --- Decide which cluster column to use for cluster-level matching ---
    if 'cluster_t' in df.columns:
        cluster_col = 'cluster_t'
    elif 'target' in df.columns and df['target'].str.contains(' ').any():
        # prefer cluster_t if available, otherwise use target
        cluster_col = 'target'
    else:
        cluster_col = 'target'

    # Normalize cluster names for matching: compare a simplified form so "Mast cells" and "Mast_cells" match
    def norm(x):
        if pd.isna(x):
            return x
        return str(x).strip().lower().replace('_', ' ').replace('-', ' ')

    df['_cluster_norm'] = df[cluster_col].map(norm)

    # Normalize provided target_cluster list
    if target_cluster is not None:
        tgt_norm = [norm(x) for x in target_cluster]
    else:
        tgt_norm = None

    # --- Incoming (to target cluster) and outgoing (from target cluster) selection ---
    if tgt_norm is not None:
        data_t = df[df['_cluster_norm'].isin(tgt_norm)].copy()   # interactions incoming to the target cluster (receptor = target)
        data_s = df[df['_cluster_norm'].isin(tgt_norm)].copy()   # interactions where cluster == target cluster (we will use gene_A as downstream)
    else:
        # if no specific target cluster, use all
        data_t = df.copy()
        data_s = df.copy()

    # Optional cluster filtering for source/target fields if provided
    if ligand_cluster is not None:
        lc_norm = [norm(x) for x in ligand_cluster]
        # assume ligand cluster corresponds to the 'source' cluster column if present, else try to filter on source text
        if 'source' in df.columns:
            data_t = data_t[data_t['source'].map(norm).isin(lc_norm)]
    if receptor_cluster is not None:
        rc_norm = [norm(x) for x in receptor_cluster]
        # assume receptor cluster corresponds to cluster_col (target)
        data_t = data_t[data_t['_cluster_norm'].isin(rc_norm)]

    # Limit to top-N by LRScore (do separately so we don't accidentally drop downstream mapping)
    data_t = data_t.nlargest(min(threshold, len(data_t)), 'LRScore') if not data_t.empty else data_t
    data_s = data_s.nlargest(min(threshold, len(data_s)), 'LRScore') if not data_s.empty else data_s

    if data_t.empty:
        print("No incoming interactions found (data_t is empty).")
    if data_s.empty:
        print("No outgoing interactions found (data_s is empty).")
    if data_t.empty and data_s.empty:
        return

    # --- Build downstream ligand mapping ---
    # Prefer using an existing downstream column if present (e.g. 'downstream_l' or 'downstream')
    if 'downstream_l' in df.columns:
        downstream_col = 'downstream_l'
    elif 'downstream' in df.columns:
        downstream_col = 'downstream'
    else:
        # fallback: use gene_A from rows where cluster==target cluster, append |DL
        downstream_col = None

    if downstream_col:
        # use cluster_col -> downstream unique mapping
        mapping = (data_s[[cluster_col, downstream_col]]
                   .dropna(subset=[downstream_col])
                   .groupby(cluster_col)[downstream_col]
                   .unique()
                   .to_dict())
    else:
        # create downstream from gene_A
        tmp = data_s[[cluster_col, 'gene_A']].dropna(subset=['gene_A'])
        tmp['downstream_l'] = tmp['gene_A'].astype(str) + "|DL"
        mapping = tmp.groupby(cluster_col)['downstream_l'].unique().to_dict()

    # Normalize keys in mapping for robust lookup
    mapping_norm = {norm(k): v for k, v in mapping.items() if pd.notna(k)}

    # Attach downstream_l to data_t by cluster (use normalized cluster)
    data_t = data_t.copy()
    data_t['_cluster_key'] = data_t[cluster_col].map(norm)
    data_t['downstream_l'] = data_t['_cluster_key'].map(lambda k: mapping_norm.get(k, []))

    # Expand rows with multiple downstream ligands (robust to NumPy arrays)
    data_t['downstream_l'] = data_t['downstream_l'].apply(
        lambda x: list(x) if isinstance(x, (list, tuple, np.ndarray))
        else ([x] if pd.notna(x) else [])
    )
    data_t = data_t.explode('downstream_l')
    data_t['downstream_l'] = data_t['downstream_l'].replace('', np.nan)
    # Expand rows with multiple downstream ligands
    data_t = data_t.assign(downstream_l=data_t['downstream_l'].apply(lambda x: list(x) if (isinstance(x, (list, tuple)) or pd.notna(x)) else []))
    data_t = data_t.explode('downstream_l')
    # If empty string or nan, set to None so gen_sankey's dropna removes them
    data_t['downstream_l'] = data_t['downstream_l'].replace('', np.nan)

    # --- If downstream_l remains NaN for all rows, print diagnostics ---
    n_down = data_t['downstream_l'].notna().sum()
    print(f"Found {n_down} non-null downstream ligand rows after expansion (out of {len(data_t)} rows).")

    # --- Build final dataframe for plotting ---
    # We'll keep columns: source, gene_A (ligand), gene_B (receptor), cluster_col (target cluster label), downstream_l
    plot_df = data_t.rename(columns={cluster_col: 'target_cluster'})[['source', 'gene_A', 'gene_B', 'target_cluster', 'downstream_l', 'LRScore']].copy()

    # If no downstream entries, still include plot but warn
    if plot_df['downstream_l'].isna().all():
        print("Warning: no downstream ligands available to draw the last Sankey layer. Check mapping or ensure downstream column exists.")
    else:
        print("Downstream ligand examples:", plot_df['downstream_l'].dropna().unique()[:10])

    # cat columns
    cat_cols = ['source', 'gene_A', 'gene_B', 'target_cluster', 'downstream_l']
    gen_sankey(plot_df, cat_cols, 'LRScore', title=plt_name or "Sankey Plot (fixed)")

# --- Keep your existing gen_sankey function but ensure it accepts labels with spaces/characters ---
# (If you changed gen_sankey earlier, the one you have should still work. If not, use your original.)


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.colors as pc
import plotly.graph_objects as go
from adjustText import adjust_text
from gprofiler import GProfiler
from sankeyflow import Sankey
import json

def plot_sankey(lrobj_tbl, target=None, target_cluster=None,
                ligand_cluster=None, receptor_cluster=None,
                plt_name=None, threshold=50):
    """
    Sankey: source → ligand → receptor → target → downstream_ligand (|DL)
    """
    print(lrobj_tbl)
    # --- Helper to ensure inputs are list-like ---
    def ensure_list(x):
        if x is None:
            return None
        return [x] if isinstance(x, str) else x

    target_cluster = ensure_list(target_cluster)
    ligand_cluster = ensure_list(ligand_cluster)
    receptor_cluster = ensure_list(receptor_cluster)

    # --- Keep only Ligand–Receptor pairs ---

    # --- Incoming to target cluster ---
    data_t = lrobj_tbl[lrobj_tbl['target'].isin(target_cluster)].copy()

    # --- Outgoing from target cluster ---
    data_s = lrobj_tbl[lrobj_tbl['source'].isin(target_cluster)].copy()

    # Optional cluster filtering
    if ligand_cluster is not None:
        data_t = data_t[data_t['source'].isin(ligand_cluster)]
    if receptor_cluster is not None:
        data_t = data_t[data_t['target'].isin(receptor_cluster)]

    # Limit to top-N by LRScore
    data_t = data_t.nlargest(min(threshold, len(data_t)), 'LRScore')
    data_s = data_s.nlargest(min(threshold, len(data_s)), 'LRScore')

    if data_t.empty and data_s.empty:
        print("No valid interactions found.")
        return

    # --- Map downstream ligands (with |DL suffix to separate layer) ---
    # --- Prepare downstream ligands (vectorized) ---
    data_s = data_s[['source', 'gene_A', 'LRScore']].copy()
    data_s = data_s[data_s['source'].isin(receptor_cluster)]
    data_s["LRScore"] = 0
    data_s['downstream_l'] = data_s['gene_A'] + "|DL"
    data_s.rename(columns={'source': 'target'}, inplace=True)  # so we can merge on target

    # Attach downstream ligands per target cluster
    downstream_map = data_s.groupby('target')['downstream_l'].unique().to_dict()

    df_expanded = data_t.copy()
    df_expanded['downstream_l'] = df_expanded['target'].map(
        lambda t: ','.join(downstream_map.get(t, []))
    )
    df_expanded = (
        df_expanded
        .assign(downstream_l=df_expanded['downstream_l'].str.split(','))
        .explode('downstream_l')
    )
    # Merge with data_t to attach all downstream ligands
    #df_expanded = data_t.merge(
    #    data_s[['target', 'downstream_l']],
    #    on='target',
    #    how='left',
    #    suffixes=('', '_down')
    #)

    #if data_t['downstream_l'].isna().all():
    #    print("No downstream ligands found for target cluster.")
    #else:
    #    n_found = data_t['downstream_l'].notna().sum()
    #    print(f"Downstream ligands added for {n_found} targets.")

    # --- Define columns and plot ---
    cat_cols = ['source', 'gene_A', 'gene_B', 'target', 'downstream_l']
    value_col = 'LRScore'  # for the main edges; use LRScore_down for downstream if desired
    gen_sankey(df_expanded, cat_cols, value_col, title=plt_name or "Sankey Plot")


def gen_sankey(df, cat_cols, value_col, title='Sankey Diagram'):
    """
    Core Sankey generator using Sankey.infer_nodes()
    """

    # Build flow list
    flows = []
    for i in range(len(cat_cols) - 1):
        pairs = df[[cat_cols[i], cat_cols[i + 1], value_col]].dropna()
        for _, row in pairs.iterrows():
            flows.append((row[cat_cols[i]], row[cat_cols[i + 1]], row[value_col], {}))

    if not flows:
        print("No valid flows for Sankey.")
        return

    # Normalize colors
    vmin, vmax = df[value_col].min(), df[value_col].max()
    limit = max(abs(vmin), abs(vmax))
    cmap = cm.get_cmap('RdBu_r')
    norm = mcolors.TwoSlopeNorm(vmin=-limit, vcenter=0, vmax=limit)

    # Apply colors to flows
    for i, (s, t, v, meta) in enumerate(flows):
        meta['color'] = mcolors.to_hex(cmap(norm(v)))
        meta['alpha'] = 0.5
        flows[i] = (s, t, 1, meta)  # uniform flow width for readability

    # --- Infer clean node layout ---
    nodes = Sankey.infer_nodes(flows)
    nodes_new = []
    for level in nodes:
        level_new = []
        for node in level:
            node_new = node + [{'color' : 'black',
                                'label_pos':'center', 'label_opts': dict(fontsize=10, bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='white'))}]
            level_new.append(node_new)
        nodes_new.append(level_new)


    fig, ax = plt.subplots(figsize=(14, 8))
    s = Sankey(
        flows=flows,
        nodes=nodes_new,
        flow_color_mode_alpha=0.3,
        node_opts=dict(label_format='{label}'),
    )
    s.draw(ax=ax)

    # --- Add colorbar ---
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.01, shrink=0.5)
    cbar.set_label(value_col, fontsize=12)

    ax.text(x=-0.05, y=1.02, s="Source", fontsize=10)
    ax.text(x=0.95, y=1.02, s="Ligand", fontsize=10)
    ax.text(x=1.95, y=1.02, s="Receptor", fontsize=10)
    ax.text(x=2.95, y=1.02, s="Target", fontsize=10)
    ax.text(x=-3.95, y=1.02, s="Downstream L", fontsize=10)
  
    ax.set_title(title)
    plt.show()

In [ ]:
#OLD STUFF
df_full_inter_tmp, top_terms_list_inter_tmp, subsetmerged_inter_tmp, subsetmerged_RCC_inter_tmp, subsetmerged_MF_inter_tmp = GO_stuff(opt = "unique", opt2="a2", opt3= "inter", which_os = "windows")


In [ ]:
subsetmerged_MF_inter

In [ ]:
df_full_intra

In [ ]:
for j, i in zip(unique_per_cluster, unique_per_cluster_unique):
    diff = (i - j).union(j - i)
    same = i.intersection(j)
    print(diff)
    #print(same)

In [ ]:
#MF

df_full_bp = df_full[df_full["Namespace"] == "MF"]
df_full_bp = df_full_bp[["Gene", "GOName"]]
df_full_bp
#df_full_bp.pivot(index="GOName", columns="Gene")
genes_bps = df_full_bp.groupby("GOName")["Gene"].apply(lambda x: list(set(x)))

#get bp terms that only appear in one single cluster
unique_per_cluster = []

#the <2 not important here, just didnt delete bc i copied it from the unique genes function
for i, cluster in enumerate(top_terms_list_MF):
    if len(clusters[i]) < 2:
        unique_per_cluster.append([])
        continue
    set_i = set(cluster)  # interactions in this cluster
    others = set().union(*[top_terms_list_MF[j]for j in range(len(top_terms_list_MF)) if j != i])
    unique = set_i - others  # LRs only in this cluster
    unique_per_cluster.append(unique)

for idx, uniq in enumerate(unique_per_cluster):
    cluster_key = list(cluster_dict.keys())[idx]
    print(f"{cluster_key}: {uniq}")
    

genes_bps_sub = genes_bps[genes_bps.index.isin(flatten(unique_per_cluster))]
print(genes_bps_sub)

#doesnt check if lr pair is in cluster if using methods 1c where genes are already split
#können einfach einzeln da sein
method= "d"

interactions_in_bp = {}
interaction_genes_only = {}


for i, lr_sets in enumerate(all_interactions):
    if not (lr_sets):
        continue
    cluster_hits = []
    interaction_genes = []
    #print(unique_per_cluster[i])
    for j in unique_per_cluster[i]:
        
        bps_of_interest = [j]

        #print(bps_of_interest)
        genes_bps_sub = genes_bps[genes_bps.index.isin(bps_of_interest)]
        #print(genes_bps_sub)
        
        
        for interaction in lr_sets:
            # split lr into ligand and receptor
            if "_" in interaction:
                genes = interaction.split("_")
            else:
                genes = [interaction]
            #print(genes)
            for bp, genes_in_bp in genes_bps_sub.items():
                if not isinstance(genes_in_bp, set):
                    genes_in_bp = set(genes_in_bp)

                #print(genes_in_bp)
                # are both l and r in this biological process
                matching = [g for g in genes if g in genes_in_bp]
                #matching = [(g, "ligand" if idx==0 else "receptor") 
                #     for idx, g in enumerate(genes) if g in genes_in_bp]
                #print(matching)
                # if both ligand and receptor are in this BP then keep
                if method == "d":
                    if len(matching) >= 1:  
                        cluster_hits.append((clusters[i], interaction, bp, matching))
                        interaction_genes.append(interaction)

                else:
                    if len(matching) >= 2:  
                        cluster_hits.append((clusters[i], interaction, bp, matching))
                        interaction_genes.append(interaction)

    interactions_in_bp[i] = cluster_hits
    interaction_genes_only[i] = interaction_genes



In [ ]:
for i, j in zip(unique_per_cluster_set, unique_per_cluster):
    diff = (i - j).union(j - i)
    print(diff)


In [ ]:
#not just unique bps but all or common over x number of clusters

#split BP & MF

#builds a dictionary mapping each GO term to the set of clusters it appears in
go_to_clusters = {}
for cluster, series in genes_bps_sep.items():
    for go_term in series.index:
        go_to_clusters.setdefault(go_term, set()).add(cluster)
print(go_to_clusters)

#filtering for no. of clusterrs
valid_go_terms = {go for go, clusters in go_to_clusters.items() if len(clusters) <= 1}
print(valid_go_terms)

#OR filtering manually?
#valid_go_terms = {go for go, clusters in go_to_clusters.items() if in go_terms_to_use}

genes_bps_sep_filtered = {
    cluster: series[series.index.isin(valid_go_terms)]
    for cluster, series in genes_bps_sep.items()
}

genes_only = {
    cluster: sorted(set(gene for genes in series.values for gene in genes))
    for cluster, series in genes_bps_sep_filtered.items()
}

In [ ]:
#'collagen binding involved in cell-matrix adhesion': {3, 4},
#'fibroblast proliferation': {5}

In [ ]:
big_df_filtered = pd.DataFrame()
for i, thing in genes_bps_sep_filtered.items():
    #print(thing)
    dfff_filtered = pd.DataFrame(data = thing)
    dfff_filtered["cluster"] = i
    big_df_filtered = pd.concat((big_df_filtered, dfff_filtered))
big_df_filtered

big_df = pd.DataFrame()
for i, thing in genes_bps_sep.items():
    print(thing)
    dfff = pd.DataFrame(data = thing)
    dfff["cluster"] = i
    big_df = pd.concat((big_df, dfff))
big_df

In [ ]:
#prints last interaction genes only, so can be either BP or MF depending on what i last ran
#for i, val in interaction_genes_only.items():
#    print(len(set(val)))

#for the cluster no. = n function
for i, val in genes_only.items():
    print(len(set(val)))

In [ ]:
RCC_lrscoress = RCC_lrscores[["LRScore", "lr_pair", "source", "target"]]
RCC_lrscoress

In [ ]:
subsetmerged = merged[merged["same_direction"] == True].drop_duplicates()
subsetmerged

In [ ]:

trend_true


In [ ]:
for name, i in cluster_dict.items():
    opt3 = "intra"

    if opt3 == "intra":
        subset_RCC = subsetmerged_RCC[(subsetmerged_RCC["cluster_s"] == name) & (subsetmerged_RCC["cluster_t"] == name)]
        subset_MF = subsetmerged_MF[(subsetmerged_MF["cluster_s"] == name) & (subsetmerged_MF["cluster_t"] == name)]  
    elif opt3 == "inter":
        subset_RCC = subsetmerged_RCC[~((subsetmerged_RCC["cluster_s"] == name) & (subsetmerged_RCC["cluster_t"] == name))]
        subset_MF = subsetmerged_MF[~((subsetmerged_MF["cluster_s"] == name) & (subsetmerged_MF["cluster_t"] == name))]  
        
          
    #print(subset_RCC)
    subsetmerged_RCC_s = subset_RCC[subset_RCC["source"].isin(i)] 
    subsetmerged_RCC_t = subset_RCC[subset_RCC["target"].isin(i)]

                                      
    subsetmerged_MF_s = subset_MF[subset_MF["source"].isin(i)]
    subsetmerged_MF_t = subset_MF[subset_MF["target"].isin(i)]

    if opt3 == "intra":
        print("RCC")
        pycrosstalker.pl.plot_sankey(subsetmerged_RCC_s,threshold = 50, plt_name = f"RCC {name}")
        print("MF")
        pycrosstalker.pl.plot_sankey(subsetmerged_MF_s, threshold=50, plt_name = f"MF {name}")

    elif opt3 == "inter":
        print("RCC")
        pycrosstalker.pl.plot_sankey(subsetmerged_RCC_s,threshold = 100, plt_name = f"RCC {name}")
        pycrosstalker.pl.plot_sankey(subsetmerged_RCC_t,threshold = 100, plt_name = f"RCC  {name}")
        print("MF")
        pycrosstalker.pl.plot_sankey(subsetmerged_MF_s, threshold=100, plt_name = f"MF {name}")
        pycrosstalker.pl.plot_sankey(subsetmerged_MF_t, threshold=100, plt_name = f"MF {name}")

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.colors as pc
import plotly.graph_objects as go
from plotnine import *
from adjustText import adjust_text
from gprofiler import GProfiler
from sankeyflow import Sankey



def plot_sankey(lrobj_tbl, target=None, ligand_cluster=None, receptor_cluster=None,
                plt_name=None, threshold=50, tfflag=True):
    """
    Generate celltype–ligand–receptor–celltype Sankey diagram.

    This function selected genes sankey plot

     Parameters
    ----------
    lrobj_tbl :
        LRobject table with all data

    target :
        gene

    ligand_cluster :
        Ligand Clusters

    receptor_cluster :
        Receptor Clusters

    plt_name :
        plot title

    threshold :
        top_n n value
    
    Returns
    -------
    Python default plot

    """


    lrobj_tbl = lrobj_tbl[(lrobj_tbl['type_gene_A'] == "Ligand") & (lrobj_tbl['type_gene_B'] == "Receptor")]
    print(lrobj_tbl)

    lrobj_tbl['gene_A'] = lrobj_tbl['gene_A'].astype(str) + '|L'
    lrobj_tbl['gene_B'] = lrobj_tbl['gene_B'].astype(str) + '|R'

    if target is not None:
        if len(target.split('|')) > 1:
            target_type = str(target.split('|')[1])
            if target_type == 'R':
                if lrobj_tbl['gene_B'].str.contains('\\|').any():
                    pass
                else:
                    target = target.split('|')[0]
                data = lrobj_tbl[lrobj_tbl['gene_B'] == target]
            elif target_type == 'L':
                if lrobj_tbl['gene_A'].str.contains('\\|').any():
                    pass
                else:
                    target = target.split('|')[0]
                data = lrobj_tbl[lrobj_tbl['gene_A'] == target]
        else:
            data = lrobj_tbl[lrobj_tbl['allpair'].str.contains(target)]
    else:
        data = lrobj_tbl

    
    if ligand_cluster is not None:
        data = data[data['source'].isin(ligand_cluster)]
    
    if receptor_cluster is not None:
        data = data[data['target'].isin(receptor_cluster)]

    color_palette = ['#00BFC4', '#FF3E3E']

    
    if len(data) >= 1:
        cat_cols = ['source', 'gene_A', 'gene_B', 'target']
        value_cols = 'LRScore'

        if len(data) < threshold:
                data = data.loc[data['LRScore'].abs().nlargest().index]
                
        data = data.loc[data['LRScore'].abs().nlargest(min(len(data), threshold)).index]
        title = plt_name

        gen_sankey(data, cat_cols, value_cols, title)
    
    else:
        print(f"Gene->{target} Not Found")
    

def gen_sankey2(df, cat_cols=[], value_cols='', title='Sankey Diagram'):
    """
    Helper function to the function plot_sankey()

     Parameters
    ----------
    df :
        Dataframe

    cat_cols :
        Columns interested in the sankey plot

    value_cols :
        Sankey plot generated using connections based on this value_cols

    title :
        Title of Sankey plot
    
    Returns
    -------
    Nothing (plots Sankey plot)

    """

    df['source'] += 'S'
    df['target'] += 'T'
    
    labelList = []
    for catCol in cat_cols:
        labelListTemp =  list((df[catCol].values))
        labelList = labelList + labelListTemp    
        
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        # sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))

    for i, label in enumerate(labelList):
        if label[-1:] == 'S' or label[-1:] == 'T':
            labelList[i] = labelList[i][:-1]
        
    norm = mcolors.Normalize(vmin=min(sourceTargetDf['count']), vmax=max(sourceTargetDf['count']))
    colormap = cm.get_cmap('RdBu_r')
    link_colors = [mcolors.to_hex(colormap(norm(value))) for value in sourceTargetDf['count']]
    
    fig =  go.Figure(data = [go.Sankey(
        node = dict(
          pad = 0,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = "white"
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = [abs(i) for i in sourceTargetDf['count']],
          color = link_colors
        )    
    )])
    
    colorbar_trace = go.Scatter(
        x=[None], y=[None], mode='markers',
        marker=dict(
            colorscale='RdBu_r',
            cmin=min(sourceTargetDf['count']),
            cmax=max(sourceTargetDf['count']),
            colorbar=dict(
                title="Value",
                thickness=15,
                len=0.5,
                x=1.05,
                xref="paper"
            )
        ),
        hoverinfo='none'
    )

    fig.add_trace(colorbar_trace)

    fig.add_annotation(x=0, y=1.05, yref="paper", text="Source", showarrow=False, font=dict(size=10))
    fig.add_annotation(x=0.33, y=1.05, yref="paper", text="Ligand", showarrow=False, font=dict(size=10))
    fig.add_annotation(x=0.66, y=1.05, yref="paper", text="Receptor", showarrow=False, font=dict(size=10))
    fig.add_annotation(x=1, y=1.05, yref="paper", text="Target", showarrow=False, font=dict(size=10))

    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[0,1]),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[0,1]),
        plot_bgcolor='white',
        autosize=True,
        width = None,
        height = 600,
        title = title,
        font = dict(size=10)
        )
    
    fig.show(config={"responsive": True})

def gen_sankey(df, cat_cols=[], value_cols='', title='Sankey Diagram', ligand_cluster = "ligand"):
    """
    Helper function to the function plot_sankey()

     Parameters
    ----------
    df :
        Dataframe

    cat_cols :
        Columns interested in the sankey plot

    value_cols :
        Sankey plot generated using connections based on this value_cols

    title :
        Title of Sankey plot
    
    Returns
    -------
    Nothing (plots Sankey plot)

    """


    # df['source'] += 'S'
    df['target'] += ' '
    
    labelList = []
    for catCol in cat_cols:
        labelListTemp =  list((df[catCol].values))
        labelList = labelList + labelListTemp    
 
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])


    vmin = sourceTargetDf['count'].min()
    vmax = sourceTargetDf['count'].max()
    limit = max(abs(vmin), abs(vmax))
    vcenter = 0
    norm = mcolors.TwoSlopeNorm(vmin=-limit, vcenter=vcenter, vmax=limit)
    # norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    cmap = plt.get_cmap('RdBu_r')
    sourceTargetDf['hex_color'] = sourceTargetDf['count'].apply(lambda x: mcolors.to_hex(cmap(norm(x))))
    #df_fixed = preprocess_cellchat_sankey(sourceTargetDf)
    #print(df_fixed)

    flows = []
    for i, row in sourceTargetDf.iterrows():
        flows.append((row['source'], row['target'], 1, {'color': row['hex_color']}))


    nodes = Sankey.infer_nodes(flows)
    nodes = list(filter(lambda empt: empt, nodes))
    print(flows)
    print(nodes)
    nodes_new = []
    for level in nodes:
        level_new = []
        for node in level:
            node_new = node + [{'color' : 'black',
                                'label_pos':'center', 'label_opts': dict(fontsize=10, bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', 
                                facecolor='white'))}]
            level_new.append(node_new)
        nodes_new.append(level_new)
    print(nodes_new)

    fig, ax = plt.subplots(figsize=(15, 10))
    s = Sankey(flows=flows,
               nodes=nodes_new,
               flow_color_mode_alpha=0.3,
               node_opts=dict(label_format='{label}')
    )
    s.draw(ax=ax)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.01, shrink=0.5)
    cbar.set_label(value_cols, fontsize=12)

    ax.text(x=-0.05, y=1.02, s="Source", fontsize=10)
    ax.text(x=0.95, y=1.02, s="Ligand", fontsize=10)
    ax.text(x=1.95, y=1.02, s="Receptor", fontsize=10)
    ax.text(x=2.95, y=1.02, s="Target", fontsize=10)

    plt.savefig(f'{title}_sankey.pdf')
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

In [ ]:

for i in cluster_dict.values():
    subsetmerged_RCC_s = subsetmerged_RCC[subsetmerged_RCC["source"].isin(i)]
    subsetmerged_RCC_t = subsetmerged_RCC[subsetmerged_RCC["target"].isin(i)]
                                          
    subsetmerged_MF_s = subsetmerged_MF[subsetmerged_MF["source"].isin(i)]
    subsetmerged_MF_t = subsetmerged_MF[subsetmerged_MF["target"].isin(i)]


    print("RCC")
    plot_sankey(subsetmerged_RCC_s,threshold = 50, plt_name = f"RCC")
    plot_sankey(subsetmerged_RCC_t,threshold = 50, plt_name = f"RCC")
    print("MF")
    plot_sankey(subsetmerged_MF_s, threshold=50, plt_name = f"MF")
    plot_sankey(subsetmerged_MF_t, threshold=50, plt_name = f"MF")

In [ ]:
subsetmerged_RCC_s

In [ ]:
subsetmerged_RCC_s

In [ ]:
subsetmerged_RCC


In [ ]:
subsetmerged_RCC

In [ ]:
# split neg and pos GO input

cluster_dict = {
    k: [v.removeprefix("RCC_").removeprefix("MF_") for v in vals]
    for k, vals in cluster_dict.items()
}

mapping = {
    item: cluster
    for cluster, items in cluster_dict.items()
    for item in items
}

grouped_list_RCC = []
grouped_list_MF = []
# Apply to each dataframe in RCC_df_list

RCC_split = RCC_diff_df.copy()
RCC_split["cluster_t"] = RCC_split["target"].map(mapping)
RCC_split["cluster_s"] = RCC_split["source"].map(mapping)


grouped_RCC_split = RCC_split.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
    "LRScore": "mean"
    })

MF_split = MF_diff_df.copy()
MF_split["cluster_t"] = MF_split["target"].map(mapping)
MF_split["cluster_s"] = MF_split["source"].map(mapping)


grouped_MF_split = MF_split.groupby(["cluster_s", "cluster_t", "lr_pair"], as_index=False).agg({
    "LRScore": "mean"
    })

RCC_split_final = grouped_RCC_split.rename(columns={"LRScore": "LRScore_RCC"})
MF_split_final = grouped_MF_split.rename(columns={"LRScore": "LRScore_MF"})

RCC_split_final["direction_RCC"] = RCC_split_final["LRScore_RCC"].apply(lambda x: "pos" if x > 0 else "neg")
MF_split_final["direction_MF"] = MF_split_final["LRScore_MF"].apply(lambda x: "pos" if x > 0 else "neg")

In [ ]:
RCC_split_final_up = RCC_split_final[RCC_split_final["direction_RCC"] == "pos"]
RCC_split_final_down = RCC_split_final[RCC_split_final["direction_RCC"] == "neg"]

MF_split_final_up = MF_split_final[MF_split_final["direction_MF"] == "pos"]
MF_split_final_down = MF_split_final[MF_split_final["direction_MF"] == "neg"]

In [ ]:
RCC_split_GO = RCC.copy()
RCC_split_GO["cluster_t"] = RCC_split_GO["target"].map(mapping)
RCC_split_GO["cluster_s"] = RCC_split_GO["source"].map(mapping)

MF_split_GO = MF.copy()
MF_split_GO["cluster_t"] = MF_split_GO["target"].map(mapping)
MF_split_GO["cluster_s"] = MF_split_GO["source"].map(mapping)


In [ ]:
RCC_df_up = RCC_split_GO.merge(
    RCC_split_final_up[["cluster_s", "cluster_t", "lr_pair"]],
    on=["cluster_s", "cluster_t", "lr_pair"],
    how="inner"
)

RCC_df_down = RCC_split_GO.merge(
    RCC_split_final_down[["cluster_s", "cluster_t", "lr_pair"]],
    on=["cluster_s", "cluster_t", "lr_pair"],
    how="inner"
)

MF_df_up = MF_split_GO.merge(
    MF_split_final_up[["cluster_s", "cluster_t", "lr_pair"]],
    on=["cluster_s", "cluster_t", "lr_pair"],
    how="inner"
)

MF_df_down = MF_split_GO.merge(
    MF_split_final_down[["cluster_s", "cluster_t", "lr_pair"]],
    on=["cluster_s", "cluster_t", "lr_pair"],
    how="inner"
)

In [ ]:
#copy for split RCC and MF
#1a,b,c(set) 2a,b,c(threshold) 3a,b,c(unique)  get all interactions
#replace c with LR-L(downstream)
#d is the same as c but is returning source and target genes together (fix that receptor complexes still need to be split)
#add empty lists so that singleton clusters dont repeat genes

def get_interactions_set_split(opt = "set", opt2 = "a", opt3 = "up"):
    all_interactions_list = []
    all_interactions_source_list = []
    all_interactions_target_list = []
    unique_per_cluster = []
    unique_per_cluster_source = []
    unique_per_cluster_target = []
    test_clust = []
    
    if opt3 == "up":
        RCC = RCC_df_up
        MF = MF_df_up

    elif  opt3 == "down":
        RCC = RCC_df_down
        MF = MF_df_down

    for i in cluster_dict: 
        tst_clust = []
        all_interactions = []
        all_interactions_source = []
        all_interactions_target = []
        
        #for clust in cluster_dict[i]:
        #    if any(clust in vals for vals in stable_partners.values()):
        #        tst_clust.append(clust)

        for clust in cluster_dict[i]:
            tst_clust.append(clust)
            
        print(tst_clust)
        tst_clust = list(map(lambda x:  x.removeprefix("RCC_").removeprefix("MF_"), tst_clust))

        if opt2 == "a" or opt2 == "b":
            clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust)
            clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust)

        if opt2 == "c":
            clust_dict_source_RCC, clust_dict_target_RCC = build_dicts(RCC, tst_clust)
            clust_dict_source_MF, clust_dict_target_MF = build_dicts(MF, tst_clust)

            downstreamL_RCC, _ = build_dicts_associated_gene(RCC, tst_clust)
            downstreamL_MF, _ = build_dicts_associated_gene(MF, tst_clust)

        if opt2 == "d" or opt2 == "d2":
            clust_dict_source_RCC, clust_dict_target_RCC = build_dicts_associated_gene(RCC, tst_clust)
            clust_dict_source_MF, clust_dict_target_MF = build_dicts_associated_gene(MF, tst_clust)

        RCC_source_sets =  {cl: set(clust_dict_source_RCC[cl]) for cl in tst_clust if len(clust_dict_target_RCC[cl]) != 0}
        MF_source_sets = {cl: set(clust_dict_source_MF[cl]) for cl in tst_clust if len(clust_dict_target_MF[cl]) != 0}

        RCC_target_sets =  {cl: set(clust_dict_target_RCC[cl]) for cl in tst_clust if len(clust_dict_target_RCC[cl]) != 0}
        MF_target_sets = {cl: set(clust_dict_target_MF[cl]) for cl in tst_clust if len(clust_dict_target_MF[cl]) != 0}

        if opt == "set" or opt == "unique":

            if opt2 == "c":

                downstreamL_RCC = {cl: set(downstreamL_RCC[cl]) for cl in tst_clust if len(downstreamL_RCC[cl]) != 0}
                downstreamL_MF = {cl: set(downstreamL_MF[cl]) for cl in tst_clust if len(downstreamL_MF[cl]) != 0}
                #print(downstreamL_RCC)

                RCC_target_sets= {
                cl: RCC_target_sets.get(cl, set()) | downstreamL_RCC.get(cl, set())
                    for cl in downstreamL_RCC.keys() if len(downstreamL_RCC[cl]) != 0}
                #print(downstreamL_RCC)

                MF_target_sets = {
                cl: MF_target_sets.get(cl, set()) | downstreamL_MF.get(cl, set())
                for cl in downstreamL_MF.keys()  if len(downstreamL_MF[cl]) != 0}
                
                #RCC_target_sets = {k: set(v) for k, v in RCC_target_sets.items()}
                #MF_target_sets = {k: set(v) for k, v in MF_target_sets.items()}
                

            RCC_source_sets_interactions = []
            RCC_target_sets_interactions = []
            RCC_total_interactions = []
            if len(RCC_target_sets) >= 1: 
                RCC_source_sets_interactions =  set.intersection(*RCC_source_sets.values())
                RCC_target_sets_interactions =  set.intersection(*RCC_target_sets.values())
                RCC_total_interactions = RCC_source_sets_interactions.union(RCC_target_sets_interactions)
                #print(RCC_target_sets)
                #print(RCC_target_sets_interactions)

            MF_source_sets_interactions = []
            MF_target_sets_interactions = []
            MF_total_interactions = []
            if len(MF_target_sets) >= 1: 
                MF_source_sets_interactions =  set.intersection(*MF_source_sets.values())
                MF_target_sets_interactions =  set.intersection(*MF_target_sets.values())
                MF_total_interactions = MF_source_sets_interactions.union(MF_target_sets_interactions)

            if opt2 == "a" or opt2 =="d":
                if len(RCC_total_interactions) > 1 and len(MF_total_interactions) > 1: 
                    all_interactions = RCC_total_interactions.intersection(MF_total_interactions)
                else:
                    all_interactions = set()
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

            if opt2 == "b"  or opt2 == "d2":
                if len(MF_source_sets_interactions) > 1 or len(RCC_source_sets_interactions) > 1: 
                    all_interactions_source = RCC_source_sets_interactions.intersection(MF_source_sets_interactions)
                    all_interactions_target = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)
                else:
                    all_interactions = set()

                all_interactions_source_list.append(all_interactions_source)
                all_interactions_target_list.append(all_interactions_target)
                test_clust.append(tst_clust)
                
            if opt2 == "c":
                if len(MF_target_sets_interactions) > 1 and len(RCC_target_sets_interactions) > 1: 
                    all_interactions = RCC_target_sets_interactions.intersection(MF_target_sets_interactions)  
                else:
                    all_interactions = set()
                
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

        ###########################################################################################################  
        if opt == "threshold":

            if opt2 == "c":

                downstreamL_RCC = {cl: set(downstreamL_RCC[cl]) for cl in tst_clust if len(downstreamL_RCC[cl]) != 0}
                downstreamL_MF = {cl: set(downstreamL_MF[cl]) for cl in tst_clust if len(downstreamL_MF[cl]) != 0}
                #print(downstreamL_RCC)

                RCC_target_sets= {
                cl: RCC_target_sets.get(cl, set()) | downstreamL_RCC.get(cl, set())
                    for cl in downstreamL_RCC.keys() if len(downstreamL_RCC[cl]) != 0}
                #print(downstreamL_RCC)

                MF_target_sets = {
                cl: MF_target_sets.get(cl, set()) | downstreamL_MF.get(cl, set())
                for cl in downstreamL_MF.keys()  if len(downstreamL_MF[cl]) != 0}
                
                #RCC_target_sets = {k: set(v) for k, v in RCC_target_sets.items()}
                #MF_target_sets = {k: set(v) for k, v in MF_target_sets.items()}
                
            RCC_source_sets_list = list(RCC_source_sets.values())
            MF_source_sets_list = list(MF_source_sets.values())
            RCC_target_sets_list = list(RCC_target_sets.values())
            MF_target_sets_list = list(MF_target_sets.values())

            threshold = 0.8  # 80%

            source_interactions_RCC = threshold_intersection(RCC_source_sets_list, threshold) if len(RCC_source_sets_list) > 0 else set()
            target_interactions_RCC = threshold_intersection(RCC_target_sets_list, threshold) if len(RCC_target_sets_list) > 0 else set()

            source_interactions_MF = threshold_intersection(MF_source_sets_list, threshold) if len(MF_source_sets_list) > 0 else set()
            target_interactions_MF = threshold_intersection(MF_target_sets_list, threshold) if len(MF_target_sets_list) > 0 else set()

            all_interactions_RCC = source_interactions_RCC.union(target_interactions_RCC) if len(source_interactions_RCC) > 0 else set()
            all_interactions_MF = source_interactions_MF.union(target_interactions_MF) if len(source_interactions_MF) > 0 else set()

            if opt2 == "a" or opt2 == "d":
                all_interactions = all_interactions_RCC.intersection(all_interactions_MF) if (len(all_interactions_MF) > 0 & len(all_interactions_RCC)) else set()
                #all_interactions = list(threshold_intersection([all_interactions_RCC, all_interactions_MF]) if len(all_interactions_MF) > 0 else set())
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)
        
            if opt2 == "b" or opt2 == "d2":
                if len(source_interactions_MF) > 1 or len(source_interactions_RCC) > 1: 
                    all_interactions_source = source_interactions_RCC.intersection(source_interactions_MF)
                    all_interactions_target = target_interactions_RCC.intersection(target_interactions_MF)
                else:
                    all_interactions = set()

                all_interactions_source_list.append(all_interactions_source)
                all_interactions_target_list.append(all_interactions_target)
                test_clust.append(tst_clust)

            if opt2 =="c":
                if len(target_interactions_MF) > 1 and len(target_interactions_RCC) > 1: 
                    all_interactions = target_interactions_RCC.intersection(target_interactions_MF)  
                else:
                    all_interactions = set()
                
                all_interactions_list.append(all_interactions)
                test_clust.append(tst_clust)

     
    if opt == "unique":

            if opt2 == "a" or opt2 == "d" or opt2 == "c":
                for i, clustergenes in enumerate(all_interactions_list):
                    #print(len(cluster))
                    if len(clustergenes) > 0:
                        #print("clustergenes", clustergenes)
                        set_i = set(clustergenes)  # interactions in this cluster
                        #print("set_i", set_i)
                        others = set().union(*[all_interactions_list[j] for j in range(len(all_interactions_list)) if j != i])
                        unique = set_i - others  # LRs only in this cluster
                        #print("unique", unique)
                        unique_per_cluster.append(unique)
                    else:
                        unique_per_cluster.append(set())

            if opt2 == "b" or opt2 == "d2":
                 
                for i, cluster in enumerate(all_interactions_source_list):
                    set_i = set(cluster)  # interactions in this cluster
                    others = set().union(*[all_interactions_source_list[j] for j in range(len(all_interactions_source_list)) if j != i])
                    unique = set_i - others  # LRs only in this cluster
                    unique_per_cluster_source.append(unique)

                for i, cluster in enumerate(all_interactions_target_list):
                    set_i = set(cluster)  # interactions in this cluster
                    others = set().union(*[all_interactions_target_list[j] for j in range(len(all_interactions_target_list)) if j != i])
                    unique = set_i - others  # LRs only in this cluster
                    unique_per_cluster_target.append(unique)


    if opt2 == "a" or opt2 =="c" or opt2 == "d":
        if opt == "set" or opt == "threshold":        
            return all_interactions_list, test_clust
        if opt == "unique":
            return unique_per_cluster, test_clust
        
    if opt2 == "b" or opt2 == "d2":
        if opt == "set" or opt == "threshold":        
            return all_interactions_source_list, all_interactions_target_list, test_clust
        if opt == "unique":
            return unique_per_cluster_source, unique_per_cluster_target, test_clust
        
    

In [ ]:
tst, clusters_cleaned = get_interactions_set_split(opt="set", opt2="c", opt3= "down")

In [ ]:
tst

In [ ]:
all_interactions = tst
df_full = pd.DataFrame()
top_terms_list = []

for i in range(len(all_interactions)):
    markers_tst = list(all_interactions[i])

    s = "_"
    clust_name = s.join(clusters[i]).replace("/", "_")
    print(clust_name)
    print(clusters[i])
    pre_genes = []
    for m in markers_tst:
        splitt = m.split("_")
        pre_genes.append(splitt)

    genes = []
    for xs in pre_genes:
        for x in xs:
            genes.append(x)
            
    genes = list(np.unique(genes))

    mg = mygene.MyGeneInfo()

    res = mg.querymany(
        genes,
        scopes="symbol",
        fields="entrezgene,ensembl.gene,go",
        species="human"
    )

    for r in res:
        print(r["query"], "→", r.get("entrezgene"))


    rows = []
    for r in res:
        geneid = r.get("entrezgene")
        genename = r.get("symbol", r["query"]) 
        if geneid is None or "go" not in r:
            continue
        gos = r["go"]
        if isinstance(gos, dict):
            for ns, terms in gos.items():  # ns = BP, MF, CC
                if isinstance(terms, list):
                    for t in terms:
                        rows.append({"Gene": genename, "GeneID": geneid, "GOID": t["id"], "Namespace": ns, "GOName": t["term"]})
                elif isinstance(terms, dict):
                    rows.append({"Gene": genename, "GeneID": geneid, "GOID": terms["id"], "Namespace": ns, "GOName": terms["term"]})

    df = pd.DataFrame(rows)

    if not df.empty:
         df = df[df["GOID"].isin(allowed_terms)]
         df = df[~df["GOName"].isin(filtered_out_terms)]
         df = df[df["GOName"].str.contains("pathway")==False]
         df = df[df["GOName"].str.contains("cardiac")==False]
         df = df[df["GOName"].str.contains("thymic")==False]

    df_full = pd.concat((df_full, df))

    if df.empty:
        top_terms_list.append(set())
        continue
    go_counts = df.groupby(["Namespace", "GOID", "GOName"]).count().rename(columns={"GeneID": "GeneCount"}).reset_index()


    for ns in ["BP"]:#, "MF"]:#, "CC"]:
     
        top_terms = go_counts[go_counts.Namespace==ns].sort_values("GeneCount", ascending=False).head(30)
        
        plt.figure(figsize=(10,6))
        sns.barplot(
            data=top_terms,
            y="GOName",
            x="GeneCount",
            palette="viridis"
        )
        top_terms_list.append(top_terms["GOName"])

        plt.xlabel("Number of Genes")
        plt.ylabel(f"GO {ns} Terms")
        #plt.title(f"{clust_name}")
        plt.tight_layout()
        #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1a_common_LRs_cluster_all_LR_pairs_unchanged/{clust_name}_top_20_BPs_GO.png")
        #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1b_common_LRs_cluster_separate_incoming_outgoing/incoming/{clust_name}_top_20_BPs_GO.png")
        #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1b_common_LRs_cluster_separate_incoming_outgoing/outgoing/{clust_name}_top_20_BPs_GO.png")
        #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform/incoming/{clust_name}_top_20_BPs_GO.png")
        #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform/outgoing/{clust_name}_top_20_BPs_GO.png")
        #plt.savefig(f"/home/larissa/Documents/Masterarbeit/plots/GO/1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform/set/RBERVertex/{clust_name}_top_20_BPs_GO.png")
        #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\1c_common_LRs_cluster_only_genes_associated_with_celltypes_GO_konform\\set\\RBERVertex\\unique\\{clusters[i][0]}_top_20_BPs_GO.png")
        #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\res025\\unique\\{clusters[i][0]}_top_20_BPs_GO.png")
        #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\res031\\unique_c\\{clusters[i][0]}_top_20_BPs_GO.png")
        #plt.savefig(f"D:\\studium\\Masterarbeit\\plots\\GO\\res031\\unique_d\\{clusters[i][0]}_top_20_BPs_GO.png")
        plt.show()


In [ ]:
df_full_bp = df_full[df_full["Namespace"] == "BP"]
df_full_bp = df_full_bp[["Gene", "GOName"]]
df_full_bp
#df_full_bp.pivot(index="GOName", columns="Gene")
genes_bps = df_full_bp.groupby("GOName")["Gene"].apply(lambda x: list(set(x)))

#get bp terms that only appear in one single cluster
unique_per_cluster = []

#the <2 not important here, just didnt delete bc i copied it from the unique genes function
for i, cluster in enumerate(top_terms_list):
    if len(clusters[i]) < 2:
        unique_per_cluster.append([])
        continue
    set_i = set(cluster)  # interactions in this cluster
    others = set().union(*[top_terms_list[j]for j in range(len(top_terms_list)) if j != i])
    unique = set_i - others  # LRs only in this cluster
    unique_per_cluster.append(unique)

for idx, uniq in enumerate(unique_per_cluster):
    print(f"Cluster {idx}: {uniq}")
    

genes_bps_sub = genes_bps[genes_bps.index.isin(flatten(unique_per_cluster))]
print(genes_bps_sub)

#doesnt check if lr pair is in cluster if using methods 1c where genes are already split
#können einfach einzeln da sein
method= "d"

interactions_in_bp = {}
interaction_genes_only = {}


for i, lr_sets in enumerate(all_interactions):
    if not (lr_sets):
        continue
    cluster_hits = []
    interaction_genes = []
    #print(unique_per_cluster[i])
    for j in unique_per_cluster[i]:
        
        bps_of_interest = [j]

        #print(bps_of_interest)
        genes_bps_sub = genes_bps[genes_bps.index.isin(bps_of_interest)]
        #print(genes_bps_sub)
        
        
        for interaction in lr_sets:
            # split lr into ligand and receptor
            if "_" in interaction:
                genes = interaction.split("_")
            else:
                genes = [interaction]
            #print(genes)
            for bp, genes_in_bp in genes_bps_sub.items():
                if not isinstance(genes_in_bp, set):
                    genes_in_bp = set(genes_in_bp)

                #print(genes_in_bp)
                # are both l and r in this biological process
                matching = [g for g in genes if g in genes_in_bp]
                #matching = [(g, "ligand" if idx==0 else "receptor") 
                #     for idx, g in enumerate(genes) if g in genes_in_bp]
                #print(matching)
                # if both ligand and receptor are in this BP then keep
                if method == "d":
                    if len(matching) >= 1:  
                        cluster_hits.append((clusters[i], interaction, bp, matching))
                        interaction_genes.append(interaction)

                else:
                    if len(matching) >= 2:  
                        cluster_hits.append((clusters[i], interaction, bp, matching))
                        interaction_genes.append(interaction)

    interactions_in_bp[i] = cluster_hits
    interaction_genes_only[i] = interaction_genes



In [ ]:
gene_to_terms = {}

for entry in res:
    gene_name = entry.get("query") 
    terms = set()
    
    go_data = entry.get("go", {})
    for category, items in go_data.items():
        for item in items:
            if "term" in item:
                terms.add(item["term"])
    
    gene_to_terms[gene_name] = terms

# Example output
for gene, terms in gene_to_terms.items():
    print(f"{gene}: {terms}")

In [ ]:
mg = mygene.MyGeneInfo()

res = mg.querymany(
    genes,
    scopes="symbol",
    fields="ensembl",
    species="human"
)
ensembl_ids = []
for i in range(0, len(res)):
    try:
        ensembl_ids.append(res[i]["ensembl"]["gene"])
    except TypeError:
        ensembl_ids.append(res[i]["ensembl"][0]["gene"])